In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community

In [30]:
%pip install --upgrade --quiet  langchain langchain-google-vertexai "langchain-google-community[featurestore]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.147.0 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [41]:
from google.cloud import aiplatform
import google.cloud.bigquery as bq
import langchain
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.schema import format_document
from langchain_community.document_loaders import DirectoryLoader, TextLoader,BigQueryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore
from datetime import datetime
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore



PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [42]:
# Define our query
query = """
SELECT id,media_type,content,test_metadata 
FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
"""

# Load the data
loader = BigQueryLoader(
    query, metadata_columns=["id"], page_content_columns=["content","media_type","test_metadata"]
)


/var/tmp/ipykernel_710571/1683864506.py:8: LangChainDeprecationWarning: The class `BigQueryLoader` was deprecated in LangChain 0.0.32 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import BigQueryLoader``.
  loader = BigQueryLoader(


In [52]:
DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}


embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)


store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


### source

https://cloud.google.com/blog/products/ai-machine-learning/rag-with-bigquery-and-langchain-in-cloud

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []
documents.extend(loader.load())
 
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5#,
   # separators=["\n\n"],
)
doc_splits = text_splitter.split_documents(documents)


# get current processing time to add it to metadata, datetime object containing current date and time
now = datetime.now()

# Add chunk number to metadata
chunk_idx=0
prev=doc_splits[0].metadata["id"]
for idx, split in enumerate(doc_splits):
    split.metadata["process_time"]=now
    if prev==split.metadata["id"]:
       split.metadata["chunk"] = chunk_idx      
    else:
        chunk_idx=0
        split.metadata["chunk"] = chunk_idx
        prev=split.metadata["id"]
    chunk_idx +=1
        

In [55]:

DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}



embedding_model = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)
bq_store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
) 

_=bq_store.add_documents(doc_splits)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


In [59]:
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore,BigQueryLoader
from datetime import datetime
import os
import logging
import json
    
def chunk_and_load():
    """
        Chunks the combination of page_content_columns from a given bigquery source string and generates
        text embeddings for them.

        Args:
            

        Returns:
             
        """
    
    project_id= os.environ.get("PROJECT_ID") 
    dataset= os.environ.get("DATASET")  
    table= os.environ.get("TABLE") 
    region= os.environ.get("REGION") 
    metadata_columns= str(os.environ.get("META_DATA_COLUMNS")).split(',') 
    page_content_columns= str(os.environ.get("PAGE_CONTENT_COLUMNS")).split(',') 
    source_query_str= os.environ.get("SOURCE_QUERY_STR") 
    separators= None if str(os.environ.get("SEPARATORS"))=="" else str(os.environ.get("SEPARATORS")).split(',') 
    chunk_size= os.environ.get("CHUNK_SIZE")
    chunk_overlap= 0 if os.environ.get("CHUNK_OVERLAP")=="" else os.environ.get("CHUNK_OVERLAP")
 

    project_id= 'nine-quality-test' 
    dataset= 'my_langchain_dataset'
    table= 'doc_and_vectors'
    region= 'us-central1'
    metadata_columns= "id".split(",")
    page_content_columns= "content,media_type,test_metadata".split(',') 
    source_query_str= """
    SELECT id,media_type,content,test_metadata 
    FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
    """
    separators= "\n\n"
    chunk_size= 25
    chunk_overlap= 5

     # Load the data
    loader = BigQueryLoader(
        source_query_str, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )

    documents = []
    documents.extend(loader.load())
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)

    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")


    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()

    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["id"]
    rows_to_insert=[]
    for idx, split in enumerate(doc_splits):
        split.metadata["process_time"]=now
        if prev==split.metadata["id"]:
           split.metadata["chunk"] = chunk_idx      
        else:
            chunk_idx=0
            split.metadata["chunk"] = chunk_idx
            prev=split.metadata["id"]
        chunk_idx +=1

        rows_to_insert.append(
                           {"id": split.metadata["id"], 
                               "process_time":split.metadata["process_time"].isoformat(),
                               "content":  split.page_content,
                               "chunk": split.metadata["chunk"]
                              }
                                     )
    logging.info (f"Metadata added to chunks")

    from google.cloud import bigquery

    client = bigquery.Client()
    table_id = f"{project_id}.{dataset}.{table}"

 
    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print("New rows have been added.")
    else:
        print(f"Encountered errors: {errors}")
    
    
    
#     logging.info (f"Bigquery store info is set -  ProjectID {project_id}, Region {region}, Dataset {dataset}, Table {table}")


#     _=bq_store.add_documents(doc_splits)
    
#     logging.info (f"Chunks and embeddings added to the store")

    return 'done'

# if __name__ == "__main__":
#     project_id= os.environ.get("PROJECT_ID") 
#     dataset= os.environ.get("DATASET")  
#     table= os.environ.get("TABLE") 
#     region= os.environ.get("REGION") 
#     metadata_columns= str(os.environ.get("META_DATA_COLUMNS")).split(',') 
#     page_content_columns= str(os.environ.get("PAGE_CONTENT_COLUMNS")).split(',') 
#     source_query_str= os.environ.get("SOURCE_QUERY_STR") 
#     separators= None if str(os.environ.get("SEPARATORS"))=="" else str(os.environ.get("SEPARATORS")).split(',') 
#     chunk_size= os.environ.get("CHUNK_SIZE")
#     chunk_overlap= 0 if os.environ.get("CHUNK_OVERLAP")=="" else os.environ.get("CHUNK_OVERLAP")
 

#     project_id= 'nine-quality-test' 
#     dataset= 'my_langchain_dataset'
#     table= 'doc_and_vectors'
#     region= 'us-central1'
#     metadata_columns= "id".split(",")
#     page_content_columns= "content,media_type,test_metadata".split(',') 
#     source_query_str= """
#     SELECT id,media_type,content,test_metadata 
#     FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
#     """
#     separators= "\n\n"
#     chunk_size= 25
#     chunk_overlap= 5
    
    
#     message=chunk_and_load(project_id=project_id, dataset=dataset, table=table, region=region,\
#                         metadata_columns= metadata_columns, page_content_columns=page_content_columns, \
#                         source_query_str=source_query_str, separators=separators, chunk_size=chunk_size, \
#                        chunk_overlap=chunk_overlap)
#     return(message)
     

In [12]:
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.4 MB/s eta 0:00:0000:01


In [17]:
import googleapiclient.discovery


def list_sub_directories(bucket_name, prefix):
    """Returns a list of sub-directories within the given bucket."""
    service = googleapiclient.discovery.build('storage', 'v1')

    req = service.objects().list(bucket=bucket_name, prefix=prefix, delimiter='/')
    
    res = req.execute()
    
    return res['prefixes']

# For the example (gs://abc/xyz), bucket_name is 'abc' and the prefix would be 'xyz/'
#print(list_sub_directories(bucket_name='raw_nine_files/sub_dir', prefix=''))

In [20]:
list(list_sub_directories(bucket_name=bucket, prefix=prefix))

['2023/', '2024/']

In [24]:
bucket='raw_nine_files'
prefix=''
l=[]
for main in list(list_sub_directories(bucket_name=bucket, prefix=prefix)):
    for dircty in list(list_sub_directories(bucket_name=bucket, prefix=main )):
       for sub_dircty in list(list_sub_directories(bucket_name=bucket, prefix=dircty )):
           print(sub_dircty)
        

2023/1/a/
2023/1/b/
2023/2/c/
2024/3/d/


In [416]:
def list_gcs_objects(bucket_name):
    """List all objects in a Google Cloud Storage bucket and print their metadata."""
    # Initialize a storage client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.get_bucket(bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix="vlt_video_extract")

    print(f"Objects in bucket '{bucket_name}':")
    for blob in blobs:
        print(f"Name: {blob.name}")
        print(f"Size: {blob.size} bytes")
        print(f"Content Type: {blob.content_type}")
        print(f"Media Link: {blob.public_url}")  # Get public URL if the object is publicly accessible
        print(f"Created: {blob.time_created}")
        print(f"Updated: {blob.updated}")
        print(f"Updated: {blob.self_link}")
        print(f"Updated: {blob.get}")
        #print(dir(blob))
        
        print("------")
    
 

In [20]:
get_video_duration("raw_nine_files","vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4")

5634.0

In [ ]:
https://ffcfb4b138e4ddd2456a59167c84bf4021642dff6ad6de28e57d52e-apidata.googleusercontent.com/download/storage/v1/b/raw_nine_files/o/2024%2F3%2Fd%2Fscreenshot_directrycontent.png?jk=AdvOr8tEXuKMrGk7hapfYy4ZPHk1yEDCUJKOHuCrx660bHpMtnBM9ujMQYZYlGC3uD9x2FfhMgvBZNcuZcVX6z301E7Zm2RjL3pJkSIu8paY66HCL6H6SLrZ4R1eCjkaxareY7JnsKrMof1amj819hbOcCMgyGfuRm74dLtW501kSwMiUwyd_6yYs6EeoeG38UVmQBiioRHQgCSMdDBu3Ej3hAwXvcIPMfCIhqvjQD7uD1OFecUTJx40xk_o_Yiy-uMA5XNsHRBiPtZ3KnUphFw5BIxArUX3DDMPFpEySCHtPIb67btv3H-FXW8MvV_1HBkw3V-xlDt3lKggVN4WRq3IeS6cF0xFD25jpDSYA-jBKL6f_U0wprpd3918-ldVEE77DysLlD7EnhIn_KErSpWi9qYWnsvqftvC3R-4iJUkehhUjtUiefJrA0uEJsZoSEeevzM_0jwtMY2Rb4LCzroTIYLDwRc4n45ILr13Rev_ATTsSx6ccGzxhfRClVaHrBRY7udek7EXIPJAx8m8MkyQCfKScX4GoWQqmPhn3hNd0LuW44r5xpO-ljrOzsBndeswe0gtwM53ns9ObCMznjoJo3DeGKFxKxsIUxmoSRaLP4bpvz7YgZEIqvrS8Mt1ZxHEXG98_meWlYyPd3A2ir7G7ZLQcxiIBm7-IKms2rmapPc3u5bM69KhjQNR93HYRxCXGzQnIhR8HYXzj9P1_6INOw3UVkBdIYX-jEFsNhkzHHTEIqrmDW_76xQ_wSrr_dOh3Oi8Ri12DXaXksMFBvbJhUGmgETi0QKfqd8EI-ReT3BbZVChbAIJ1RNUe-wd7guT8mKdpxTicWK0KpMnGaU3DL8tlY9GZwpFyRDmJRHwBP7iqVjU4M4jF2KyHwdL589yyxlheVn-fmItOag0vPxkysf4pX0qL52WNP3ch15tN2D8LePFp4S__rQwXD-vdsciDDXBH3DOeLeKWR4-h6lVkYxi6XtHVbLgOE26lurqe1Q0XQQcJYA1usDNdJUNYwuW1qYGiB7VH6PDYSrSHuVj2YB3bmsGxpxOf2vxrhhJZzIhKU3Yxy-BX-tXp0AK8C4oowhu66feVTx6dIl_d3xQlOinsq5kEbsxvb9aSSKPkuNkfFLTbksJhW2-QWHnv4Bwtf7uC6zMLxVfocy-iCZAMTkGa9icvrHJeT3htPvpTQYbHq_kOeTVZUj3Hw-LRcaUQnxQfOTLVWmGXBAyWhWr28l5zblWxYxGMXMP&isca=1

In [6]:
#video_url = "https://storage.cloud.google.com/raw_nine_files/vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4"
video_url="https://ffba87f3998dc58bc084d165a1de6442fbc82dae22183352ca8b1bc-apidata.googleusercontent.com/download/storage/v1/b/raw_nine_files/o/vlt_video_extract%2FMAAT%2FMAAT2024_1_A_HBB.mp4?jk=AdvOr8stvY3hRXD9psznNOnR68vfY46qQuSRwB7uO7udZdeU2LGGz-_54PmfT-yobQGxdyq1OJd1dnjB-KHuYc64Ky3hZ45dZPi3LocJ9kOpF2E7EAfzzbh1tnu855pcL63SAw-jIOe4AKQdfRQcXxO-KhuhEkUxHY9jthAzHPVv7wYH5zPsrKLuKIZzlfqJ-kh2KYLi-HlC9SDa9FtEPZbHXSuJZvlUXZK9exW_8Qnxlo5EtNLP386be6siB0rndlMgdKeHPTcO3r7RpXPlfeFm6hQuWDpe-vAW6wRz-BJzlnXOc1-cdXQ7_2Hz85JrdQ6SO-7Sf9fG7Nhk-wSLWGf_9hzaD4AaaVqhCJqmojJGQ1NID5iLry10QGX6r2Ce23Rp0P0dicWb0ePc6BygOVOPM2NZZWyHQO0OJB63aaXsz5MAFTA7dU0oQsz36-d4NzwcEiIfb4Sv86r5uowgeS_pBBOJrv0Y3NlmF1BJ_Eq5bN-q10S0wJ040NMQNs-fWfDKSSCBd-gWovKWk16r4DLSOvt10-EVfNWs8qHEnI9VJB_3E03XxhrONQxX2VqQoCnPMYcAob1lbfezAefR9dpy6xoXvcoB_GajqVoBBLlF6lk8cX69YxsL_gdSzNVJIbRMwVYfdipLKhXxiipUJM3GuB-OlD9OwKjpbYK2hGCxN0GR_pY5mrRREka8FAsxMpZvmEXEgqwYKwMPRCHmtfRj4uAyubVUPP-WCBEQQ_0r8VG1IBFqhLLrM4rqrWwxM5Vejjo6NPS6QP2Va5sOfptdcyLqTXpaNBesHkrO_hyC4ClOtwkhx1-x8rTPHat8VkUHUcPGHug2aVHp8l-zzTs-puLcF4N24hfkD06MVqbUWslkKHXTUiEbRiIOgg854ObQ976NsZI8GPeDjIIbCYzCZyDGUR9lYcJ98oQlopCEtPZsKrwUE-nV40WvQ2ABXGrW3szWrOlaOPQDwmwCTTSgWvlyxWlqI8txdIV3B1pccav3bkOiqlz7e2BcBc6rxaxu_P4Xy9_ZBGggkFO7dIiBizs0O6kkWoFib5985-bXbEvXNdeClyhuTSG9JML5-EiQqMvfm0_uW69Y9LtAUw9QwlMg6AP9nZytgw1aPBTDJlU6nZHM4ARrS7-Tv9QV3eUIGANS466_mAPP-M5wR3sWIy8fq7LhUCWmVxOqCt-iMgiqP6ZgyyRd6AgY1wAk-lshrySf1uDZV5xgeQrp8WhfNS19HNMexHSqreqPYIs&isca=1"
try:
    clip = VideoFileClip(video_url)
    print("Video Duration:", clip.duration)
except Exception as e:
    print("Error:", e)

Video Duration: 5634.0


In [ ]:
from google.cloud import storage
from google.oauth2 import service_account
from moviepy.editor import VideoFileClip
import io

 
# Initialize the storage client
client = storage.Client( )

# Set your bucket and file names
bucket_name = 'raw_nine_files'
object_name = 'vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4'

# Create a file-like object from GCS
bucket = client.bucket(bucket_name)
blob = bucket.blob(object_name)

# Download the blob as a byte stream
video_stream = io.BytesIO()
blob.download_to_file(video_stream)
video_stream.seek(0)  # Reset stream position to the beginning

# Use moviepy to get the video length from the stream
try:
    clip = VideoFileClip(video_stream)
    print("Video Duration (seconds):", clip.duration)
except Exception as e:
    print("Error:", e)

In [418]:
# Call the function with your bucket name
bucket_name = 'raw_nine_files'
list_gcs_objects(bucket_name)

Objects in bucket 'raw_nine_files':
Name: vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4
Size: 3613937486 bytes
Content Type: video/mp4
Media Link: https://storage.googleapis.com/raw_nine_files/vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4
Created: 2024-10-07 01:41:14.024000+00:00
Updated: 2024-10-07 01:41:14.024000+00:00
Updated: https://www.googleapis.com/storage/v1/b/raw_nine_files/o/vlt_video_extract%2FMAAT%2FMAAT2024_1_A_HBB.mp4
Updated: /b/raw_nine_files/o/vlt_video_extract%2FMAAT%2FMAAT2024_1_A_HBB.mp4
------
Name: vlt_video_extract/MAAT/MAAT2024_21_A_HBB.mp4
Size: 2748590138 bytes
Content Type: video/mp4
Media Link: https://storage.googleapis.com/raw_nine_files/vlt_video_extract/MAAT/MAAT2024_21_A_HBB.mp4
Created: 2024-10-07 01:41:14.209000+00:00
Updated: 2024-10-07 01:41:14.209000+00:00
Updated: https://www.googleapis.com/storage/v1/b/raw_nine_files/o/vlt_video_extract%2FMAAT%2FMAAT2024_21_A_HBB.mp4
Updated: /b/raw_nine_files/o/vlt_video_extract%2FMAAT%2FMAAT2024_21_A_HBB.mp4
-----

In [73]:
from google.cloud import storage

def list_gcs_objects(bucket_name):
    """List all objects in a Google Cloud Storage bucket."""
    # Initialize a client for interacting with GCS
    storage_client = storage.Client()

    # Get the bucket by name
    bucket = storage_client.get_bucket(bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs()

    print(f"Objects in bucket {bucket_name}:")
    for blob in blobs:
        blob = bucket.get_blob(blob.name)

        print(blob.mediaLink)

# Call the function with your bucket name
bucket_name = 'raw_nine_files'
list_gcs_objects(bucket_name)

Objects in bucket raw_nine_files:


AttributeError: 'Blob' object has no attribute 'mediaLink'

In [95]:
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore,BigQueryLoader
from datetime import datetime
import os
import logging
import json
    
def chunk_and_load(project_id: str= None, dataset: str= None, table: str= None, region: str =None,\
                        metadata_columns: list[str]=None, page_content_columns: list[str]= None, \
                        source_query_str: str= None, separators:  list[str]=None, chunk_size: int=None, \
                       chunk_overlap: int=0):
    """
        Chunks the combination of page_content_columns from a given bigquery source string and generates
        text embeddings for them.

        Args:
            

        Returns:
             
        """


     # Load the data
    loader = BigQueryLoader(
        source_query_str, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )

    documents = []
    documents.extend(loader.load())
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)
    
   
    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")


    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()

    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["id"]
    rows_to_insert=[]
    for idx, split in enumerate(doc_splits):
        split.metadata["process_time"]=now
        if prev==split.metadata["id"]:
           split.metadata["chunk"] = chunk_idx      
        else:
            chunk_idx=0
            split.metadata["chunk"] = chunk_idx
            prev=split.metadata["id"]
        chunk_idx +=1

        rows_to_insert.append(
                           {"id": split.metadata["id"], 
                               "process_time":split.metadata["process_time"].isoformat(),
                               "content":  split.page_content,
                               "chunk": split.metadata["chunk"]
                              }
                                     )
    logging.info (f"Metadata added to chunks")

    from google.cloud import bigquery

    client = bigquery.Client()
    table_id = f"{project_id}.{dataset}.{table}"

 
    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print("New rows have been added.")
    else:
        print(f"Encountered errors: {errors}")
    
    
    
#     logging.info (f"Bigquery store info is set -  ProjectID {project_id}, Region {region}, Dataset {dataset}, Table {table}")


#     _=bq_store.add_documents(doc_splits)
    
#     logging.info (f"Chunks and embeddings added to the store")

    return 'done'

if __name__ == "__main__":
    project_id= os.environ.get("PROJECT_ID") 
    dataset= os.environ.get("DATASET")  
    table= os.environ.get("TABLE") 
    region= os.environ.get("REGION") 
    metadata_columns= str(os.environ.get("META_DATA_COLUMNS")).split(',') 
    page_content_columns= str(os.environ.get("PAGE_CONTENT_COLUMNS")).split(',') 
    source_query_str= os.environ.get("SOURCE_QUERY_STR") 
    separators= None if str(os.environ.get("SEPARATORS"))=="" else str(os.environ.get("SEPARATORS")).split(',') 
    chunk_size= 1000 if str(os.environ.get("CHUNK_SIZE"))=="None" else int(str(os.environ.get("CHUNK_SIZE")))  
    chunk_overlap= 0 if str(os.environ.get("CHUNK_OVERLAP"))=="None" else int(str(os.environ.get("CHUNK_OVERLAP")))
 
 

    project_id= 'nine-quality-test' 
    dataset= 'my_langchain_dataset'
    table= 'doc_and_vectors'
    region= 'us-central1'
    metadata_columns= "id".split(",")
    page_content_columns= "content,media_type,test_metadata".split(',') 
    source_query_str= """
    SELECT id,media_type,content,test_metadata 
    FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
    """
    separators= "\\n\\n"
    chunk_size= 25
    chunk_overlap= 5
    
    
    message=chunk_and_load(project_id=project_id, dataset=dataset, table=table, region=region,\
                        metadata_columns= metadata_columns, page_content_columns=page_content_columns, \
                        source_query_str=source_query_str, separators=separators, chunk_size=chunk_size, \
                       chunk_overlap=chunk_overlap)
    print(message)

New rows have been added.
done


In [283]:
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json


def upload_file(request_file : tempfile,dest_bucket_name:str =None,request_file_folder: str =None,request_file_prefix: str =None, version: int=0):
     temp=request_file
     client = storage.Client()
     # Extract name to the temp file
     temp_file = "".join([str(temp.name)])
     # Uploading the temp image file to the bucket
     dest_filename = f"{request_file_folder}/"+request_file_prefix+'_'+str(version)+".json" 
     dest_bucket = client.get_bucket(dest_bucket_name)
     dest_blob = dest_bucket.blob(dest_filename)
     dest_blob.upload_from_filename(temp_file)                              

                                
def create_image_request_file( dest_bucket_name: str= None, source_bucket_name: str= None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None):
    """create batch request file(s) of up to 30000 and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            str path:  path to the gcs request file object
          
    """
        

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix='2023')  
    version=0
    index=0     
    max_index=30000

    for blob in blobs:                         
                    if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         if index==0:
                            request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                            rf= open(request_file.name, "a") 
                 
                         request_list=[
                               json.dumps(
                                      {
                                        "request": 
                                             {
                                              "contents": 
                                                  {"parts": [{ "fileData": 
                                                                 {"fileUri": gcsuri, "mimeType": mimeType}
                                                              }, 
                                                              {"text": prompt_text
                                                              }
                                                            ]
                                                              , 
                                                    "role": "user"
                                                  }
                                                  , 
                                                  "generation_config": 
                                                       {"max_output_tokens": max_output_tokens, 
                                                        "temperature":temperature, 
                                                         "top_k": top_k, 
                                                         "top_p": top_p
                                                        }
                                                  , 
                                                  "safety_settings": 
                                                   [
                                                    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                                     "threshold": "BLOCK_NONE"
                                                     }
                                                   , 
                                                   {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                                   "threshold": "BLOCK_NONE"
                                                   },
                                                   {"category": "HARM_CATEGORY_HARASSMENT", 
                                                   "threshold": "BLOCK_NONE"
                                                   }, 
                                                   {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                    "threshold": "BLOCK_NONE"
                                                    }


                                                   ]
                                             }
                                      }
                               )  +"\n"
                         ]

                         rf.writelines(request_list)
                         rf.flush()
 
                         if index==(max_index-1):
                                upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version)
                                rf.close()                                
                                index=0
                                version +=1
                                request_list=[]
                                rf=None

                         else:                               
                                index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version)
 
    return 1
 
def create_batch_request_file( ):
 
  
  
    dest_bucket_name ='artifacts-nine-quality-test-embeddings' #request_args['destination_bucket']
    source_bucket_name ='raw_nine_files'# request_args['source_bucket']
    request_file_prefix = "image_request"#request_args['request_file_prefix']
    request_file_folder =  "image_batch_request_fldr"#request_args['request_file_folder']
    prompt_text= "You are an assistant tasked with summarizing images for retrieval. \\ These summaries will be embedded and used to retrieve the raw image. \\ Give a concise summary of the image that is well optimized for retrieval.\\ If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\\ If you are not sure about any info, please do not make it up. Do not add any extra text to the output."#request_args['prompt_text']
    media_types= ['image/jpeg','image/png']#list(request_args['media_type'])
    request_content= "image"#request_args['request_content']
 
    temperature=0.5
    max_output_tokens=2048
    top_p=50
    top_k=0.5

    if  request_content=='image':
      _=create_image_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k
                                      )  

    return {"status":'SUCCESS'}

create_batch_request_file()

{'status': 'SUCCESS'}

In [90]:
import functions_framework
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json


def upload_file(request_file : tempfile,dest_bucket_name:str =None,request_file_folder: str =None,request_file_prefix: str =None, version: int=0, request_file_post_fix : str=""):

    """upload file into gcs
   
        Args:
            tempfile request_file: request file
            str dest_bucket_name:  name of destination bucket
            str request_file_folder: name of the destination folder name to write files to
            list request_file_prefix: prefix of request file name
          
    """

    temp=request_file
    client = storage.Client()
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+'_'+request_file_post_fix+'_'+str(version)+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)                              

                      
def create_video_request_file( dest_bucket_name: str= None, source_bucket_name: str= None, source_folder_name: str=None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None, max_request_per_file: int =None, video_metadata_file: str=""):

    """create batch request  file(s) of up to 30000 for video and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            int : number of generated files
          
    """       

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder_name)  
    version=0
    index=0     
    max_index=max_request_per_file

    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    now=datetime.strftime(now, '%Y%m%d%H%M%S')
    versions=[]
    

    # Instantiate a Google Cloud Storage client and specify required bucket and file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(dest_bucket_name)
    blob = bucket.blob(video_metadata_file)

    # Download the contents of the blob as a string and then parse it using json.loads() method
    data = json.loads(blob.download_as_string(client=None))
    video_metadata=data['items']
    

    segments_to_process=120 #segments duration
    intervals=120#intervals
    video_start =0 #where from video to start




    for blob in blobs:                         
        if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         metadata=list(filter(lambda element: element['gcs_uri'] ==gcsuri, video_metadata))[0]
                         video_duration=metadata["videoOriginalDurationSecond"]
                         prev=video_start
                         for val in range (segments_to_process,video_duration+segments_to_process,segments_to_process):
                                offset={'start':prev, 'end':val}
                                
                                startOffset=offset['start']
                                endOffset=offset['end']
                                if endOffset>=video_duration:
                                     endOffset=video_duration
                                print(offset)
                                prev=val  
                                if index==0:
                                    request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                                    rf= open(request_file.name, "a") 
                                
                                 
                                request_list=[
                                       json.dumps(
                                              {
                                                "request": 
                                                     {
                                                      "contents":  {
                                                                        "parts": [
                                                                            {
                                                                            "fileData":  {"fileUri": gcsuri, "mimeType": mimeType},
                                                                            "videoMetadata": {
                                                                                "endOffset": {
                                                                                "nanos": 0,
                                                                                "seconds": endOffset
                                                                                },
                                                                                "startOffset": {
                                                                                "nanos": 0,
                                                                                "seconds": startOffset
                                                                                }
                                                                            }

                                                                            },
                                                                            {
                                                                            "text": prompt_text +"\n"+ f
                                                                            } 

                                                                        ],
                                                                        "role": "user"
                                                                        }
                                                          , 
                                                          "generation_config": 
                                                               {"max_output_tokens": max_output_tokens, 
                                                                "temperature":temperature, 
                                                                 "top_k": top_k, 
                                                                 "top_p": top_p
                                                                }
                                                          , 
                                                          "safety_settings": 
                                                           [
                                                            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                                             "threshold": "BLOCK_NONE"
                                                             }
                                                           , 
                                                           {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                                           "threshold": "BLOCK_NONE"
                                                           },
                                                           {"category": "HARM_CATEGORY_HARASSMENT", 
                                                           "threshold": "BLOCK_NONE"
                                                           }, 
                                                           {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                            "threshold": "BLOCK_NONE"
                                                            }


                                                           ]
                                                     }
                                              }
                                       )  +"\n"
                                 ]

                                rf.writelines(request_list)
                                rf.flush()

                                if index==(max_index-1):
                                        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                        rf.close() 
                                        versions.append(version)                               
                                        index=0
                                        version +=1
                                        request_list=[]
                                        rf=None

                                else:                               
                                        index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   
 
    return len(versions)

@functions_framework.http
def create_batch_request_file(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
#     request_json = request.get_json(silent=True)
#     request_args = request.args

#     dest_bucket_name =request_args['destination_bucket']
#     source_bucket_name =request_args['source_bucket']
#     source_folder_name=request_args['source_folder']
#     request_file_prefix =request_args['request_file_prefix']
#     request_file_folder =request_args['request_file_folder']
#     prompt_text= request_args['prompt_text']
#     media_types= [media.strip() for media in  str(request_args['media_types']).strip().replace("[",''). replace(']','').replace("'",'').split(',')]

 
#     request_content= request_args['request_content']

#     if request_args and 'temperature' in request_args:
#         temperature= request_args['temperature']
#     else:
#       temperature=0.5

#     if request_args and 'max_output_tokens' in request_args:
#        max_output_tokens= request_args['max_output_tokens'] 
#     else:
#          max_output_tokens=2048

#     if request_args and 'top_p' in request_args:
#         top_p= request_args['top_p']
#     else:
#          top_p=0.5

#     if request_args and 'top_k' in request_args:
#         top_k= request_args['top_k']
#     else:
#          top_k=50

#     if request_args and 'max_request_per_file' in request_args:
#         max_request_per_file= request_args['max_request_per_file']
#     else:
#       max_request_per_file=30000
    
    dest_bucket_name ='artifacts-nine-quality-test-embeddings' #request_args['destination_bucket']
    source_bucket_name ='raw_nine_files'# request_args['source_bucket']
    request_file_prefix = "video_request"#request_args['request_file_prefix']
    request_file_folder =  "video_batch_request_fldr"#request_args['request_file_folder']
    prompt_text= "You are an assistant tasked with summarizing images for retrieval. \\ These summaries will be embedded and used to retrieve the raw image. \\ Give a concise summary of the image that is well optimized for retrieval.\\ If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\\ If you are not sure about any info, please do not make it up. Do not add any extra text to the output."#request_args['prompt_text']
    media_types= ['video/mp4']#list(request_args['media_type'])
    request_content= "video"#request_args['request_content']
    source_folder_name="vlt_video_extract/OTHERS"
    temperature=0.5
    max_output_tokens=2048
    top_p=50
    top_k=0.5
    max_request_per_file=30000
    video_metadata_file="vlt_video_metadata_fldr/vlt_video_metadata.json"

    versions=0
    if  request_content=='image':
      versions=create_image_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k, max_request_per_file=max_request_per_file
                                      )  
    if  request_content=='video':
      versions=create_video_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k, max_request_per_file=max_request_per_file,
                                     video_metadata_file=video_metadata_file
                                      ) 

    return {"status":"SUCCESS","file_count":versions}


In [91]:
create_batch_request_file('')

{'start': 0, 'end': 120}
{'start': 120, 'end': 240}
{'start': 240, 'end': 360}
{'start': 360, 'end': 480}
{'start': 480, 'end': 600}
{'start': 600, 'end': 720}
{'start': 720, 'end': 840}
{'start': 840, 'end': 960}
{'start': 960, 'end': 1080}


{'status': 'SUCCESS', 'file_count': 1}

In [216]:
from google.cloud import storage

def move_files( dest_bucket_name: str= None, source_bucket_name: str= None,
                          source_folder: str= None, destination_folder: str= None, mime_types: list[str]= None):
    
    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs()  
 

    for blob in blobs:
        # Check if the file is an .mp4 file
        if blob.content_type in ["video/mp4"]:
        
            # Create the new blob name for the destination
            temp=blob.name.split("/")
            name=blob.name.split('/')[len(temp)-1]
            if name.startswith("SYD-NINE"):
                dest_folder= f"{destination_folder}/NINE_NIEWS"
            elif name.startswith("MAAT"):
                dest_folder= f"{destination_folder}/MAAT"
            elif name.startswith("60MI"):
                dest_folder= f"{destination_folder}/SIXTY_MINUTES"
            else:
                 dest_folder= f"{destination_folder}/OTHERS"
                     
            new_blob_name = f"{dest_folder}/{name}"
            print(new_blob_name)

            # Copy the blob to the new location
            bucket.copy_blob(blob, bucket, new_blob_name)
            print(f"Copied {blob.name} to {new_blob_name}")

# Example usage
bucket_name = 'raw_nine_files'
source_folder = ''  # Make sure to include the trailing slash
destination_folder = 'vlt_video_extract'

move_files(dest_bucket_name=bucket_name,source_bucket_name=bucket_name, source_folder=source_folder, destination_folder=destination_folder,mime_types=[])

 

vlt_video_extract/SIXTY_MINUTES/60MI23_33_A_HBB.mp4
Copied 60MI23_33_A_HBB.mp4 to vlt_video_extract/SIXTY_MINUTES/60MI23_33_A_HBB.mp4
vlt_video_extract/SIXTY_MINUTES/60MI23_33_A_HBB_fixed.mp4
Copied 60MI23_33_A_HBB_fixed.mp4 to vlt_video_extract/SIXTY_MINUTES/60MI23_33_A_HBB_fixed.mp4
vlt_video_extract/SIXTY_MINUTES/60MI24_1_A_HBB.mp4
Copied 60MI24_1_A_HBB.mp4 to vlt_video_extract/SIXTY_MINUTES/60MI24_1_A_HBB.mp4
vlt_video_extract/SIXTY_MINUTES/60MI24_6_A_HBB.mp4
Copied 60MI24_6_A_HBB.mp4 to vlt_video_extract/SIXTY_MINUTES/60MI24_6_A_HBB.mp4
vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4
Copied MAAT2024_1_A_HBB.mp4 to vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4
vlt_video_extract/MAAT/MAAT2024_21_A_HBB.mp4
Copied MAAT2024_21_A_HBB.mp4 to vlt_video_extract/MAAT/MAAT2024_21_A_HBB.mp4
vlt_video_extract/MAAT/MAAT2024_36_A_HBB.mp4
Copied MAAT2024_36_A_HBB.mp4 to vlt_video_extract/MAAT/MAAT2024_36_A_HBB.mp4
vlt_video_extract/NINE_NIEWS/SYD-NINE_NNNTW24_21_A.mp4
Copied SYD-NINE_NNNTW24_21_A.mp4 t

In [358]:
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_community import BigQueryLoader
from datetime import datetime
import logging
from google.cloud import bigquery
import os, json

def create_dataset(project_id,dataset_id,region_id):
    """
        Create a dataset
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str region_id:  name of the region under whcih the dataset should be created.               
    """
    
    client = bigquery.Client(project_id)
    # Check if the dataset exists
    try:
        dataset = client.get_dataset(f"{project_id}.{dataset_id}")  # Make an API request.
        print(f"Dataset '{dataset_id}' already exists.")
    except :
        # If the dataset does not exist, create it
        
        dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
        dataset.location = region_id # Set your desired location

        # Create the dataset
        dataset = client.create_dataset(dataset)  # Make an API request.
        print(f"Dataset '{dataset_id}' created successfully.")
        
 
def create_table(project_id,dataset_id,table_id):
    
    """
        Create a table 
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str table_id:  name of the table.
        Returns:
            list[bigquery.SchemaField] schema: table schema- list of columns
             
    """
        
    
    client = bigquery.Client(project_id)
    
    full_table_id = f"{project_id}.{dataset_id}.{table_id}"

    # Define the table schema
    schema = [
        bigquery.SchemaField("request_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("original_content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("media_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("path", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("test_metadata", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("chunk", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("process_time", "DATETIME", mode="REQUIRED"),
    ]
                

    # Check if the table exists
    try:
        table = client.get_table(full_table_id)  # Make an API request.
       
        print(f"Table '{full_table_id}' already exists.")
        
        # Truncate the table if exist
        query = f"TRUNCATE TABLE `{full_table_id}`"

        # Execute the query
        try:
            client.query(query).result()  # Make an API request.
            print(f"Table '{full_table_id}' truncated successfully.")
        except Exception as e:
            print(f"Error truncating table '{full_table_id}': {e}")
    except :
        # If the table does not exist, create it
        table = bigquery.Table(full_table_id, schema=schema)
        #add partitioning
        # table.time_partitioning = bigquery.TimePartitioning(
        #     type_=bigquery.TimePartitioningType.DAY,
        #     field="request_id",  # This field will be used for partitioning
        # )

        # Create the table
        table = client.create_table(table)  # Make an API request.
        print(f"Table '{full_table_id}' created successfully.")
        
    return schema


In [366]:
from datetime import datetime
def chunk_bq_content(request_args):
    """
        Chunks the combination of page_content_columns from a given bigquery source string and load the result into bigquery
             
    """
    status=''
 
    try:
        project_id=  request_args['project_id']
        dataset_id=  request_args['dataset']
        table= request_args['table']
        region= request_args['region']
        metadata_columns= str(request_args['metadata_columns']).split(',') 
        page_content_columns= str(request_args['page_content_columns']).split(',') 
        source_query_str= request_args['source_query_str']
        separators= None if str(request_args['separators'])=="" else str(request_args['separators']).split(',') 
        chunk_size= 1000 if str(request_args['chunk_size']) in ["None",""] else int(str(request_args['chunk_size']))  
        chunk_overlap= 0 if str(request_args['chunk_overlap']) in ["None",""] else int(str(request_args['chunk_overlap']))  
    except:        
            project_id= 'nine-quality-test' 
            dataset_id= 'langchain_dataset'
            table= 'chunked_data'
            region= 'us-central1'
            metadata_columns= [ col.strip() for col in  "id,content,path,media_type,test_metadata".split(",")]
            page_content_columns= [col.strip() for col in "content,test_metadata".split(',') ]
            source_query_str= """
            SELECT id,media_type,content,test_metadata, path
            FROM `nine-quality-test.Nine_Quality_Test.test_long_artcles` ;
            """
            separators= "\n"
            chunk_size= 500
            chunk_overlap= 10
            #return {'record_count':0, 'status':'ERROR- Set required input parameters'}



     # Load the data
    loader = BigQueryLoader(
        query=source_query_str, project=project_id, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )
    
    documents = []
    documents.extend(loader.load())
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)    
     
    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")

    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()
    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["id"]
    rows_to_insert=[]
    request_date=datetime.today().strftime('%Y_%m_%d')
    
        
    client = bigquery.Client(project_id)    
    #create data set if does not exist
    create_dataset(project_id,dataset_id,region)
    max_index=30000 #maximum number of requests in a batch
 
    max_index=2
    record_count=0
    for idx, split in enumerate(doc_splits):
            split.metadata["process_time"]=now
            if prev==split.metadata["id"]:
               split.metadata["chunk"] = chunk_idx      
            else:
                chunk_idx=0
                split.metadata["chunk"] = chunk_idx
                prev=split.metadata["id"]
            chunk_idx +=1
            version=idx // max_index
            request_id = request_date+'_'+str(version)
            rows_to_insert.append(
                               {  "request_id":  request_id  , 
                                   "id": split.metadata["id"], 
                                   "process_time":split.metadata["process_time"].isoformat(),
                                   "content":  split.page_content,
                                   "original_content": split.metadata["content"],
                                   "chunk": split.metadata["chunk"],
                                   "media_type": split.metadata["media_type"],
                                   "path": split.metadata["path"],
                                   "test_metadata": split.metadata["test_metadata"]                            

                                  }
                                         )
            
            if (idx+1) % max_index==0:
               
                #create table new if does not exist
                table=f"{dataset_id}_{request_id}"
                table_schema=create_table(project_id,dataset_id,table)
                #push the data into the table
                table_id = f"{project_id}.{dataset_id}.{table}"
                dataset  = client.dataset(dataset_id)
                table = dataset.table(table)
                job_config = bigquery.LoadJobConfig()
                job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
                job_config.schema = table_schema
                job = client.load_table_from_json(rows_to_insert, table, job_config = job_config)
                #moving to next batch
                record_count=record_count+len(rows_to_insert)
                rows_to_insert=[]
    
    return {'status':'SUCCESS', 'record_count':record_count, 'count_of_tables':version+1}

In [367]:
chunk_bq_content('')

Dataset 'langchain_dataset' already exists.
Table 'nine-quality-test.langchain_dataset.langchain_dataset_2024_10_08_0' created successfully.
LoadJob<project=nine-quality-test, location=us-central1, id=7a692df9-5ea4-44dc-b0b0-07fdf4e90f5b>
Table 'nine-quality-test.langchain_dataset.langchain_dataset_2024_10_08_1' created successfully.
LoadJob<project=nine-quality-test, location=us-central1, id=62281b95-20fd-480c-946d-79189b1475c5>


{'status': 'SUCCESS', 'record_count': 4, 'count_of_tables': 2}

In [ ]:
 
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_community import BigQueryLoader
from datetime import datetime
import logging
from google.cloud import bigquery
import os, json

def create_dataset(project_id,dataset_id,region_id):
    """
        Create a dataset
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str region_id:  name of the region under whcih the dataset should be created.               
    """
    
    client = bigquery.Client(project_id)
    print(project_id)
    # Check if the dataset exists
    try:
        dataset = client.get_dataset(f"{project_id}.{dataset_id}")  # Make an API request.
        print(f"Dataset '{dataset_id}' already exists.")
    except :
        # If the dataset does not exist, create it
        
        dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
        dataset.location = region_id # Set your desired location

        # Create the dataset
        dataset = client.create_dataset(dataset)  # Make an API request.
        print(f"Dataset '{dataset_id}' created successfully.")
        
 
def create_table(project_id,dataset_id,table_id):
    
    """
        Create a table 
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str table_id:  name of the table.
        Returns:
            list[bigquery.SchemaField] schema: table schema- list of columns
             
    """
        
    
    client = bigquery.Client(project_id)
    
    full_table_id = f"{project_id}.{dataset_id}.{table_id}"

    # Define the table schema
    schema = [
        bigquery.SchemaField("request_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("original_content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("content", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("media_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("path", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("test_metadata", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("chunk", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("process_time", "DATETIME", mode="REQUIRED"),
    ]
                

    # Check if the table exists
    try:
        table = client.get_table(full_table_id)  # Make an API request.
        print(f"Table '{full_table_id}' already exists.")
        
        # Truncate the table if exist
        query = f"DROP TABLE `{full_table_id}`"

        # Execute the query
        try:
            client.query(query).result()  # Make an API request.
            print(f"Table '{full_table_id}' dropped successfully.")
        except Exception as e:
            print(f"Error dropping table '{full_table_id}': {e}")
    except :
        # If the table does not exist, create it
        table = bigquery.Table(full_table_id, schema=schema)

        # Create the table
        table = client.create_table(table)  # Make an API request.
        print(f"Table '{full_table_id}' created successfully.")
        
    return schema


if 1==1:
    """
        Chunks the combination of page_content_columns from a given bigquery source string and load the result into bigquery
             
    """
    status=''  

 

    # project_id=  request_args['project_id']
    # dataset_id=  request_args['dataset']
    # table= request_args['table']
    # region= request_args['region']
    # metadata_columns= [col.strip() for col in  str(request_args['metadata_columns']).split(',') ]
    # page_content_columns= [col.strip() for col in str(request_args['page_content_columns']).split(',') ]
    # source_query_str= request_args['source_query_str']
    # #separators= "\n" if str(request_args['separators'])=="" else str(request_args['separators']).split(',') 
    # chunk_size= 1000 if str(request_args['chunk_size']) in ["None",""] else int(str(request_args['chunk_size']))  
    # chunk_overlap= 0 if str(request_args['chunk_overlap']) in ["None",""] else int(str(request_args['chunk_overlap'])) 
    # max_prompt_count_limit=3000 if str(request_args['max_prompt_count_limit']) in ["None",""] else int(str(request_args['max_prompt_count_limit'])) 
    # # except Exception as e: 
    if 1==1:
            project_id= 'nine-quality-test' 
            dataset_id= 'langchain_dataset'
            table= 'chunked_data'
            region= 'us-central1'
            metadata_columns= [ col.strip() for col in  "id, content, path, media_type, test_metadata".split(",")]
            page_content_columns= [col.strip() for col in "content, test_metadata".split(',') ]
            source_query_str= """
            SELECT id,media_type,content,test_metadata, path
            FROM `nine-quality-test.Nine_Quality_Test.test_long_artcles` ;
            """
            #separators= "\n"
            chunk_size= 500
            chunk_overlap= 10
            max_prompt_count_limit=2
     

            #return {'record_count':0, 'status':'ERROR- Set required input parameters '+e}



     # Load the data
    loader = BigQueryLoader(
        query=source_query_str, project=project_id, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )
    
    documents = []
    documents.extend(loader.load())
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       #separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)    
     
    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")

    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()
    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["id"]
    rows_to_insert=[]
    request_date=datetime.today().strftime('%Y_%m_%d')    
        
    client = bigquery.Client(project_id)    
    #create data set if does not exist
    create_dataset(project_id,dataset_id,region)
    max_index=max_prompt_count_limit #maximum number of requests in a batch
    record_count=0
    prefix=f"{table}_{request_date}" 
    for idx, split in enumerate(doc_splits):
            split.metadata["process_time"]=now
            if prev==split.metadata["id"]:
               split.metadata["chunk"] = chunk_idx      
            else:
                chunk_idx=0
                split.metadata["chunk"] = chunk_idx
                prev=split.metadata["id"]
                
            chunk_idx +=1
            version=idx // max_index
            request_id = request_date+'_'+str(version)
            rows_to_insert.append(
                               {  "request_id":  request_id  , 
                                   "id": split.metadata["id"], 
                                  ## "process_time":split.metadata["process_time"].isoformat(),
                                   "content":  split.page_content,
                                   "original_content": split.metadata["content"],
                                   "chunk": split.metadata["chunk"],
                                   "media_type": split.metadata["media_type"],
                                   "path": split.metadata["path"],
                                   "test_metadata": split.metadata["test_metadata"]                            

                                  }
                                         )
            
            if (idx+1) % max_index==0:
               
                #create table new if does not exist
                table=f"{prefix}_{version}"
               
                print(table)
                print('************')
                table_schema=create_table(project_id,dataset_id,table)
                #push the data into the table
                table_id = f"{project_id}.{dataset_id}.{table}"
                dataset  = client.dataset(dataset_id)
                table = dataset.table(table)
                job_config = bigquery.LoadJobConfig()
                job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
                job_config.schema = table_schema
                job = client.load_table_from_json(rows_to_insert, table, job_config = job_config)
                #moving to next batch
                record_count=record_count+len(rows_to_insert)
                rows_to_insert=[]
                print(job.job_id)
    
    #return {'status':'SUCCESS', 'record_count':record_count, 'count_of_tables':version+1, 'table_name_prefix':prefix}

In [ ]:
chunk_bq_content()

In [4]:
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json
from moviepy.editor import VideoFileClip
import datetime as date_time

import math

def upload_file(request_file : tempfile,dest_bucket_name:str =None,request_file_folder: str =None,request_file_prefix: str =None, version: int=0, request_file_post_fix : str=""):

    """upload file into gcs
   
        Args:
            tempfile request_file: request file
            str dest_bucket_name:  name of destination bucket
            str request_file_folder: name of the destination folder name to write files to
            list request_file_prefix: prefix of request file name
          
    """

    temp=request_file
    client = storage.Client()
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+'_'+request_file_post_fix+'_'+str(version)+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)                              

                                
def create_image_request_file( dest_bucket_name: str= None, source_bucket_name: str= None, source_folder_name: str=None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", max_prompt_count_limit: int=30000, temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None):

    """create batch request file(s) of up to 30000 and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            int : as number of generated request files
          
    """       

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder_name)  
    version=0
    index=0     
    max_index=max_prompt_count_limit

    now=dt.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    now=dt.strftime(now, '%Y%m%d%H%M%S')
    versions=[]

    for blob in blobs:                         
                    if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         if index==0:
                            request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                            rf= open(request_file.name, "a") 
                 
                         request_list=[
                               json.dumps(
                                      {
                                        "request": 
                                             {
                                              "contents": 
                                                  {"parts": [{ "fileData": 
                                                                 {"fileUri": gcsuri, "mimeType": mimeType}
                                                              }, 
                                                              {"text": prompt_text
                                                              }
                                                            ]
                                                              , 
                                                    "role": "user"
                                                  }
                                                  , 
                                                  "generation_config": 
                                                       {"max_output_tokens": max_output_tokens, 
                                                        "temperature":temperature, 
                                                         "top_k": top_k, 
                                                         "top_p": top_p
                                                        }
                                                  , 
                                                  "safety_settings": 
                                                   [
                                                    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                                     "threshold": "BLOCK_NONE"
                                                     }
                                                   , 
                                                   {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                                   "threshold": "BLOCK_NONE"
                                                   },
                                                   {"category": "HARM_CATEGORY_HARASSMENT", 
                                                   "threshold": "BLOCK_NONE"
                                                   }, 
                                                   {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                    "threshold": "BLOCK_NONE"
                                                    }


                                                   ]
                                             }
                                      }
                               )  +"\n"
                         ]

                         rf.writelines(request_list)
                         rf.flush()
 
                         if index==(max_index-1):
                                upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                rf.close() 
                                versions.append(version)                               
                                index=0
                                version +=1
                                request_list=[]
                                rf=None

                         else:                               
                                index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   
 
    return len(versions)



def generate_signed_url_v4(blob):
    """Generates a v4 signed URL for downloading a blob.

    Note that this method requires a service account key file. You can not use
    this if you are using Application Default Credentials from Google Compute
    Engine or from the Google Cloud SDK.
    """

    from google.auth.transport import requests
    from google.auth import default, compute_engine
    
    credentials, _ = default()
    
    # then within your abstraction
    auth_request = requests.Request()
    credentials.refresh(auth_request)
    
    signing_credentials = compute_engine.IDTokenCredentials(
        auth_request,
        "",
        service_account_email=credentials.service_account_email
    )
    url = blob.generate_signed_url(
        version="v4",
        # This URL is valid for 15 minutes
        expiration=date_time.timedelta(minutes=15),
        # Allow GET requests using this URL.
        method="GET",
         credentials=signing_credentials,
    )
    return url


def get_video_duration(signed_url):
   
  try:
    video_uri=signed_url
    
    clip = VideoFileClip(video_uri)
    duration = clip.duration
    clip.close()  # Release resources
    return duration
  except Exception as e:
    if "moov atom not found" in str(e):
      print("Error: The video file seems to be corrupted or incomplete.")
     
    else:
      print(f"Error reading video file: {e}")
    

def create_video_request_file( dest_bucket_name: str= None, source_bucket_name: str= None, source_folder_name: str=None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", max_prompt_count_limit: int=30000, temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None):

    """create batch request file(s) of up to 30000 and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            int 1: as SUCCESS
          
    """       

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder_name)  
    version=0
    index=0     
    max_index=max_prompt_count_limit

    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    now=datetime.strftime(now, '%Y%m%d%H%M%S')
    versions=[]

    for blob in blobs:                         
                    if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         signed_url= generate_signed_url_v4(blob)
                         video_duration = 2641# math.ceil(get_video_duration(signed_url))
                         print(video_duration)
                         for 
                         if index==0:
                            request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                            rf= open(request_file.name, "a") 
                 
                         request_list=[
                               json.dumps(
                                      {
                                        "request": {
                                            "contents": {
                                                "parts": [
                                                    {
                                                        "fileData": {
                                                            "fileUri": "gs://raw_nine_files/60MI23_33_A_HBB.mp4",
                                                            "mimeType": "video/mp4"
                                                        },
                                                        "videoMetadata": {
                                                            "endOffset": {
                                                                "nanos": 0,
                                                                "seconds": 240
                                                            },
                                                            "startOffset": {
                                                                "nanos": 0,
                                                                "seconds": 120
                                                            }
                                                        }
                                                    },
                                                    {
                                                        "text": "You are an assistant tasked with describing videos for retrieval.These descriptions will be embedded and used to retrieve th
                                                        e raw video.\n Chapterize the video content by grouping the video content into chapters with intervals of 120 seconds 
                                                        and providing a concise detail for each chapter that is well optimized for retrieval.\nIf there is a 
                                                        famous person like politician, celebrity or athlete, indicate their name and describe what they 
                                                        are famous for.\n Describe important scenes in the video concisely. If you are not sure about any info, 
                                                        please do not make it up. \n Only consider video from 120 seconds to 240 seconds. \nIf it is the last chapter, 
                                                        set the endOffset to 240 instead.\n For result, follow JSON schema.<JSONSchema>{\"description\":\"A list of chapters\",\"items\":{\"properties\":{\"Content\":{\"type\":\"string\"},\"endOffset\":{\"type\":\"integer\"},\"startOffset\":{\"type\":\"integer\"}},\"required\":[\"startOffset\",\"endOffset\",\"Content\"],\"type\":\"object\"},\"type\":\"array\"}</JSONSchema>"
                                                    }
                                                ],
                                                "role": "user"
                                            },
                                            "generation_config": {
                                                "max_output_tokens": 2048,
                                                "temperature": 0.5,
                                                "top_k": 40,
                                                "top_p": 0.8
                                            },
                                            "safety_settings": [
                                                {
                                                    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                                                    "threshold": "BLOCK_NONE"
                                                },
                                                {
                                                    "category": "HARM_CATEGORY_HATE_SPEECH",
                                                    "threshold": "BLOCK_NONE"
                                                },
                                                {
                                                    "category": "HARM_CATEGORY_HARASSMENT",
                                                    "threshold": "BLOCK_NONE"
                                                },
                                                {
                                                    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                    "threshold": "BLOCK_NONE"
                                                }
                                            ]
                                        }
                                    }
                               )  +"\n"
                         ]

                         rf.writelines(request_list)
                         rf.flush()
 
                         if index==(max_index-1):
                                upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                rf.close() 
                                versions.append(version)                               
                                index=0
                                version +=1
                                request_list=[]
                                rf=None

                         else:                               
                                index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   
 
    return len(versions)

def create_batch_request_file(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
    try:
        request_json = request.get_json(silent=True)
        request_args = request.args

        dest_bucket_name =request_args['destination_bucket']
        source_bucket_name =request_args['source_bucket']
        source_folder_name=request_args['source_folder']
        request_file_prefix =request_args['request_file_prefix']
        request_file_folder =request_args['request_file_folder']
        prompt_text= request_args['prompt_text']
        max_prompt_count_limit=int(str(request_args['max_prompt_count_limit']))  
        media_types= [media.strip() for media in  str(request_args['media_types']).strip().replace("[",''). replace(']','').replace("'",'').split(',')]


        request_content= request_args['request_content']

        if request_args and 'temperature' in request_args:
            temperature= request_args['temperature']
        else:
          temperature=0.5

        if request_args and 'max_output_tokens' in request_args:
           max_output_tokens= request_args['max_output_tokens'] 
        else:
             max_output_tokens=2048

        if request_args and 'top_p' in request_args:
            top_p= request_args['top_p']
        else:
             top_p=0.5

        if request_args and 'top_k' in request_args:
            top_k= request_args['top_k']
        else:
             top_k=50
    except:
            dest_bucket_name ='artifacts-nine-quality-test-embeddings' #request_args['destination_bucket']
            source_bucket_name ='raw_nine_files'# request_args['source_bucket']
            request_file_prefix = "image_request"#request_args['request_file_prefix']
            request_file_folder =  "image_batch_request_fldr"#request_args['request_file_folder']
            prompt_text= "You are an assistant tasked with summarizing images for retrieval. \\ These summaries will be embedded and used to retrieve the raw image. \\ Give a concise summary of the image that is well optimized for retrieval.\\ If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\\ If you are not sure about any info, please do not make it up. Do not add any extra text to the output."#request_args['prompt_text']
            media_types= ['image/jpeg','image/png']#list(request_args['media_type'])
            request_content= "image"#request_args['request_content']

            temperature=0.5
            max_output_tokens=2048
            top_p=50
            top_k=0.5
            request_content="video"
            media_types= ['video/mp4']
            max_prompt_count_limit=2
            source_folder_name='vlt_video_extract'
            prompt_text="You are an assistant tasked with describing videos for retrieval. \
                                Describe this video well optimized for retrieval. Mention anything important the people talk about.\
                                If there is a text or logo mention that in details. If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\
                                If you are not sure about any info, please do not make it up. Do not add any extra text to the output."

    versions=0
    if  request_content=='image':
      versions=create_image_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,max_prompt_count_limit=max_prompt_count_limit,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k
                                      )  
    elif request_content=='video':
      versions=create_video_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,max_prompt_count_limit=max_prompt_count_limit,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k
                                      )  

    return {"status":"SUCCESS","file_count":versions}


create_batch_request_file('')

5634
4285
3910
2701
2690
2980
2719
2720
2885
2641


{'status': 'SUCCESS', 'file_count': 5}

In [10]:

import gcsfs
from pymediainfo import MediaInfo

def get_video_metadata_from_stream(gcsuri):
    fs = gcsfs.GCSFileSystem()

    # Open the file stream using gcsfs
    with fs.open(gcsuri, 'rb') as video_file:
        # Use pymediainfo to extract metadata directly from the stream
        media_info = MediaInfo.parse(video_file)
        for track in media_info.tracks:
            if track.track_type == 'Video':
                return track.duration / 1000  # Convert ms to seconds

# bucket_name = 'raw_nine_files'
# source_blob_name = 'vlt_video_extract/SIXTY_MINUTES/60MI23_33_A_HBB.mp4'

# # Get video metadata (duration) from stream
# duration = get_video_metadata_from_stream(bucket_name, source_blob_name)
# print(f"Video duration: {duration} seconds")



In [11]:

def generate_video_metadata( dest_bucket_name: str= None, source_bucket_name: str= None, source_folder_name: str=None,
                          metadata_file_prefix: str= None, metadata_file_folder: str= None, mime_types: list[str]= None 
                           ):

    """ generates video length
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            int 1: as SUCCESS
          
    """       

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder_name)  
    

    for blob in blobs:                         
                    if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         video_duration=get_video_metadata_from_stream(gcsuri)
                         print(video_duration)
 
    return 1

In [37]:
video_duration=get_video_metadata_from_stream("gs://raw_nine_files/vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4")
print(video_duration)

5634.0


In [38]:
video_duration=get_video_metadata_from_stream("gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2.mp4")
print(video_duration)

1005.0


In [39]:
generate_video_metadata(   source_bucket_name="raw_nine_files", source_folder_name="vlt_video_extract",
                           mime_types=['video/mp4']
                           )

5634.0
4285.0
3910.0
2700.04
2690.0
2979.01
1005.0
2719.0
2719.04
2885.0
2641.0


1

In [33]:
import vertexai
import datetime

from vertexai.generative_models import Part
from vertexai.preview import caching

# TODO(developer): Update and un-comment below line
# PROJECT_ID = "your-project-id"

 
system_instruction = """
You are an expert researcher. You always stick to the facts in the sources provided, and never make up new facts.
Now look at these research papers, and answer the following questions.
"""

contents = [
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2312.11805v3.pdf",
        mime_type="application/pdf",
    ),
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2403.05530.pdf",
        mime_type="application/pdf",
    ),
]

cached_content = caching.CachedContent.create(
    model_name="gemini-1.5-pro-002",
    system_instruction=system_instruction,
    contents=contents,
    ttl=datetime.timedelta(days=2),
    display_name="example-cache",
)

print(cached_content.name)
# Example response:
# 1234567890

1320504668865953792


In [32]:
datetime.timedelta(days=2)

datetime.timedelta(days=2)

In [35]:
cached_content.to_dict()

{'name': 'projects/494586852359/locations/us-central1/cachedContents/1320504668865953792',
 'model': 'projects/nine-quality-test/locations/us-central1/publishers/google/models/gemini-1.5-pro-002',
 'createTime': '2024-10-10T00:05:18.148425Z',
 'updateTime': '2024-10-10T00:05:18.148425Z',
 'expireTime': '2024-10-12T00:05:18.129147Z',
 'displayName': 'example-cache'}

In [72]:
x= data['items']
x

[{'createTime': 'llm_response.body.createTime',
  'expireTime': 'llm_response.body.expireTime',
  'gcs_uri': 'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2 - Copy.mp4',
  'model': 'llm_response.body.model',
  'name': 'llm_response.body.name',
  'updateTime': 'llm_response.body.updateTime',
  'usageMetadata': 'llm_response.body.usageMetadata'},
 {'createTime': 'llm_response.body.createTime',
  'expireTime': 'llm_response.body.expireTime',
  'gcs_uri': 'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2.mp4',
  'model': 'llm_response.body.model',
  'name': 'llm_response.body.name',
  'updateTime': 'llm_response.body.updateTime',
  'usageMetadata': 'llm_response.body.usageMetadata'}]

In [77]:
l=list(filter(lambda element: element['gcs_uri'] == 'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2 - Copy.mp4', x))

In [79]:
l[0]["gcs_uri"]

'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2 - Copy.mp4'

In [99]:



# Import the Google Cloud client library and JSON library
from google.cloud import storage
import json

# Instantiate a Google Cloud Storage client and specify required bucket and file
storage_client = storage.Client()
bucket = storage_client.get_bucket('raw_nine_files')
blob = bucket.blob('test111.json')

# Download the contents of the blob as a string and then parse it using json.loads() method
data = json.loads(blob.download_as_string(client=None))
for x in data['items']:
    print(x)
    print()
    
    
data["cache_expiry_time"]

{'createTime': 'llm_response.body.createTime', 'expireTime': 'llm_response.body.expireTime', 'gcs_uri': 'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2 - Copy.mp4', 'model': 'llm_response.body.model', 'name': 'llm_response.body.name', 'updateTime': 'llm_response.body.updateTime', 'usageMetadata': 'llm_response.body.usageMetadata'}

{'createTime': 'llm_response.body.createTime', 'expireTime': 'llm_response.body.expireTime', 'gcs_uri': 'gs://raw_nine_files/vlt_video_extract/OTHERS/splitProcessOutput_vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB_360p_split2.mp4', 'model': 'llm_response.body.model', 'name': 'llm_response.body.name', 'updateTime': 'llm_response.body.updateTime', 'usageMetadata': 'llm_response.body.usageMetadata'}



'2024-10-12T23:45:42.158262Z'

In [ ]:
# Download the contents of the blob as a string and then parse it using json.loads() method
data = json.loads(blob.download_as_string(client=None))
for item in enumerate( data.items):
    print(item)

In [ ]:
         if index==0:
                            request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                            rf= open(request_file.name, "a") 
                 
                         request_list=[
                               json.dumps(
                                      {
                                        "request": {
                                            "contents": {
                                                "parts": [
                                                    {
                                                        "fileData": {
                                                            "fileUri": "gs://raw_nine_files/60MI23_33_A_HBB.mp4",
                                                            "mimeType": "video/mp4"
                                                        },
                                                        "videoMetadata": {
                                                            "endOffset": {
                                                                "nanos": 0,
                                                                "seconds": 240
                                                            },
                                                            "startOffset": {
                                                                "nanos": 0,
                                                                "seconds": 120
                                                            }
                                                        }
                                                    },
                                                    {
                                                        "text": "You are an assistant tasked with describing videos for retrieval.These descriptions will be embedded and used to retrieve the raw video.\n Chapterize the video content by grouping the video content into chapters with intervals of 120 seconds and providing a concise detail for each chapter that is well optimized for retrieval.\nIf there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\n Describe important scenes in the video concisely. If you are not sure about any info, please do not make it up. \n Only consider video from 120 seconds to 240 seconds. \nIf it is the last chapter, set the endOffset to 240 instead.\n For result, follow JSON schema.<JSONSchema>{\"description\":\"A list of chapters\",\"items\":{\"properties\":{\"Content\":{\"type\":\"string\"},\"endOffset\":{\"type\":\"integer\"},\"startOffset\":{\"type\":\"integer\"}},\"required\":[\"startOffset\",\"endOffset\",\"Content\"],\"type\":\"object\"},\"type\":\"array\"}</JSONSchema>"
                                                    }
                                                ],
                                                "role": "user"
                                            },
                                            "generation_config": {
                                                "max_output_tokens": 2048,
                                                "temperature": 0.5,
                                                "top_k": 40,
                                                "top_p": 0.8
                                            },
                                            "safety_settings": [
                                                {
                                                    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                                                    "threshold": "BLOCK_NONE"
                                                },
                                                {
                                                    "category": "HARM_CATEGORY_HATE_SPEECH",
                                                    "threshold": "BLOCK_NONE"
                                                },
                                                {
                                                    "category": "HARM_CATEGORY_HARASSMENT",
                                                    "threshold": "BLOCK_NONE"
                                                },
                                                {
                                                    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                    "threshold": "BLOCK_NONE"
                                                }
                                            ]
                                        }
                                    }
                               )  +"\n"
                         ]

                         rf.writelines(request_list)
                         rf.flush()
 
                         if index==(max_index-1):
                                upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                rf.close() 
                                versions.append(version)                               
                                index=0
                                version +=1
                                request_list=[]
                                rf=None

                         else:                               
                                index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   

In [17]:
import vertexai

from vertexai.preview import caching

# TODO(developer): Update and un-comment below lines
# PROJECT_ID = "your-project-id"
# cache_id = "your-cache-id"
 
# cached_content = caching.CachedContent(cached_content_name=cache_id)
# cached_content.delete()

for x in caching.CachedContent.list():
     x=x.name.split("/")[-1]
     print(x)
     cached_content = caching.CachedContent(cached_content_name=x)
     #cached_content.delete()

In [54]:
def get_video_duration(request: VideoRequest):
    """ generates video length
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            int 1: as SUCCESS
          
    """
    
    fs = gcsfs.GCSFileSystem()
    # Open the file stream using gcsfs
    with fs.open(gcsuri, 'rb') as video_file:
              # Use pymediainfo to extract metadata directly from the stream
              media_info = MediaInfo.parse(video_file)
              for track in media_info.tracks:
                  if track.track_type == 'Video':
                      duration= track.duration / 1000  # Convert ms to seconds
                      print(duration)
                      break

NameError: name 'VideoRequest' is not defined

In [57]:
import functions_framework
import gcsfs
from pymediainfo import MediaInfo

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

# Define the input data model
class VideoRequest(BaseModel):
    url: str  # Input: video URL

# Define the output data model
class VideoResponse(BaseModel):
    duration: float  # Output: Duration in seconds

@app.post("/get-video-duration", response_model=VideoResponse)
async def get_video_duration(request: VideoRequest):
    """
    get video duration of a given gcs url
    
    """
    
    fs = gcsfs.GCSFileSystem()
    # Open the file stream using gcsfs
    with fs.open(gcsuri, 'rb') as video_file:
              # Use pymediainfo to extract metadata directly from the stream
              media_info = MediaInfo.parse(video_file)
              for track in media_info.tracks:
                  if track.track_type == 'Video':
                      duration= track.duration / 1000  # Convert ms to seconds
                      print(duration)
                      break
 

    return VideoResponse(duration=duration)



In [102]:
import functions_framework
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json


def upload_file(request_file : tempfile,dest_bucket_name:str =None,request_file_folder: str =None,request_file_prefix: str =None, version: int=0, request_file_post_fix : str=""):

    """upload file into gcs
   
        Args:
            tempfile request_file: request file
            str dest_bucket_name:  name of destination bucket
            str request_file_folder: name of the destination folder name to write files to
            list request_file_prefix: prefix of request file name
          
    """

    temp=request_file
    client = storage.Client()
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+'_'+request_file_post_fix+'_'+str(version)+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)                              

                      
def create_video_request_file( dest_bucket_name: str= None, source_bucket_name: str= None, source_folder_name: str=None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None, max_request_per_file: int =None, video_metadata_file: str=""):

    """create batch request  file(s) of up to 30000 for video and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            int : number of generated files
          
    """       

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder_name)  
    version=0
    index=0     
    max_index=max_request_per_file

    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    now=datetime.strftime(now, '%Y%m%d%H%M%S')
    versions=[]
    

    # Instantiate a Google Cloud Storage client and specify required bucket and file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(dest_bucket_name)
    blob = bucket.blob(video_metadata_file)

    # Download the contents of the blob as a string and then parse it using json.loads() method
    data = json.loads(blob.download_as_string(client=None))
    video_metadata=data['items']
    

    segments_to_process=120 #segments duration
    intervals=120#intervals
    video_start =0 #where from video to start




    for blob in blobs:                         
        if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         metadata=list(filter(lambda element: element['gcs_uri'] ==gcsuri, video_metadata))[0]
                         video_duration=metadata["videoOriginalDurationSecond"]
                         cache_id=metadata["name"]
                         prev=video_start
                         for val in range (segments_to_process,video_duration+segments_to_process,segments_to_process):
                                offset={'start':prev, 'end':val}
                                
                                startOffset=offset['start']
                                endOffset=offset['end']
                                if endOffset>=video_duration:
                                     endOffset=video_duration
                                print(offset)
                                prev=val 
                                segment_prompt= f"Only consider video from {startOffset} seconds to {endOffset} seconds. Ignore analyzing the rest of video.\n" 
                                if index==0:
                                    request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                                    rf= open(request_file.name, "a") 
                                
                                 
                                request_list=[
                                       json.dumps(
                                              {
                                                "request": 
                                                     {
                                                     "cached_content": cache_id,
                                                      "contents":  {
                                                                        "parts": [
                                                                            {
                                                                            "fileData":  {"fileUri": gcsuri, "mimeType": mimeType},
                                                                            "videoMetadata": {
                                                                                "endOffset": {
                                                                                "nanos": 0,
                                                                                "seconds": endOffset
                                                                                },
                                                                                "startOffset": {
                                                                                "nanos": 0,
                                                                                "seconds": startOffset
                                                                                }
                                                                            }

                                                                            },
                                                                            {
                                                                            "text": prompt_text +"\n"+ segment_prompt
                                                                            } 

                                                                        ],
                                                                        "role": "user"
                                                                        }
                                                          , 
                                                          "generation_config": 
                                                               {"max_output_tokens": max_output_tokens, 
                                                                "temperature":temperature, 
                                                                 "top_k": top_k, 
                                                                 "top_p": top_p
                                                                }
                                                          , 
                                                          "safety_settings": 
                                                           [
                                                            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                                             "threshold": "BLOCK_NONE"
                                                             }
                                                           , 
                                                           {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                                           "threshold": "BLOCK_NONE"
                                                           },
                                                           {"category": "HARM_CATEGORY_HARASSMENT", 
                                                           "threshold": "BLOCK_NONE"
                                                           }, 
                                                           {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                            "threshold": "BLOCK_NONE"
                                                            }


                                                           ]
                                                     }
                                              }
                                       )  +"\n"
                                 ]

                                rf.writelines(request_list)
                                rf.flush()

                                if index==(max_index-1):
                                        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                        rf.close() 
                                        versions.append(version)                               
                                        index=0
                                        version +=1
                                        request_list=[]
                                        rf=None

                                else:                               
                                        index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   
 
    return len(versions)

@functions_framework.http
def create_batch_request_file(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
#     request_json = request.get_json(silent=True)
#     request_args = request.args

#     dest_bucket_name =request_args['destination_bucket']
#     source_bucket_name =request_args['source_bucket']
#     source_folder_name=request_args['source_folder']
#     request_file_prefix =request_args['request_file_prefix']
#     request_file_folder =request_args['request_file_folder']
#     prompt_text= request_args['prompt_text']
#     media_types= [media.strip() for media in  str(request_args['media_types']).strip().replace("[",''). replace(']','').replace("'",'').split(',')]

 
#     request_content= request_args['request_content']

#     if request_args and 'temperature' in request_args:
#         temperature= request_args['temperature']
#     else:
#       temperature=0.5

#     if request_args and 'max_output_tokens' in request_args:
#        max_output_tokens= request_args['max_output_tokens'] 
#     else:
#          max_output_tokens=2048

#     if request_args and 'top_p' in request_args:
#         top_p= request_args['top_p']
#     else:
#          top_p=0.5

#     if request_args and 'top_k' in request_args:
#         top_k= request_args['top_k']
#     else:
#          top_k=50

#     if request_args and 'max_request_per_file' in request_args:
#         max_request_per_file= request_args['max_request_per_file']
#     else:
#       max_request_per_file=30000
    
    dest_bucket_name ='artifacts-nine-quality-test-embeddings' #request_args['destination_bucket']
    source_bucket_name ='raw_nine_files'# request_args['source_bucket']
    request_file_prefix = "video_request"#request_args['request_file_prefix']
    request_file_folder =  "video_batch_request_fldr"#request_args['request_file_folder']
    prompt_text= "You are an assistant tasked with summarizing images for retrieval. \\ These summaries will be embedded and used to retrieve the raw image. \\ Give a concise summary of the image that is well optimized for retrieval.\\ If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\\ If you are not sure about any info, please do not make it up. Do not add any extra text to the output."#request_args['prompt_text']
    media_types= ['video/mp4']#list(request_args['media_type'])
    request_content= "video"#request_args['request_content']
    source_folder_name="vlt_video_extract/OTHERS"
    temperature=0.5
    max_output_tokens=2048
    top_p=50
    top_k=0.5
    max_request_per_file=30000
    video_metadata_file="vlt_video_metadata_fldr/vlt_video_metadata.json"

    versions=0
    if  request_content=='image':
      versions=create_image_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k, max_request_per_file=max_request_per_file
                                      )  
    if  request_content=='video':
      versions=create_video_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k, max_request_per_file=max_request_per_file,
                                     video_metadata_file=video_metadata_file
                                      ) 

    return {"status":"SUCCESS","file_count":versions}


In [103]:
create_batch_request_file('')

{'start': 0, 'end': 120}
{'start': 120, 'end': 240}
{'start': 240, 'end': 360}
{'start': 360, 'end': 480}
{'start': 480, 'end': 600}
{'start': 600, 'end': 720}
{'start': 720, 'end': 840}
{'start': 840, 'end': 960}
{'start': 960, 'end': 1080}


{'status': 'SUCCESS', 'file_count': 1}

In [105]:
import functions_framework
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json


def upload_file(request_file : tempfile,dest_bucket_name:str =None,request_file_folder: str =None,request_file_prefix: str =None, version: int=0, request_file_post_fix : str=""):

    """upload file into gcs
   
        Args:
            tempfile request_file: request file
            str dest_bucket_name:  name of destination bucket
            str request_file_folder: name of the destination folder name to write files to
            list request_file_prefix: prefix of request file name
          
    """

    temp=request_file
    client = storage.Client()
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+'_'+request_file_post_fix+'_'+str(version)+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)                              

def create_video_request_file( dest_bucket_name: str= None, source_bucket_name: str= None, source_folder_name: str=None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None, max_request_per_file: int =None, video_metadata_file: str=""):

    """create batch request  file(s) of up to 30000 for video and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str source_bucket_name: name of the source  gcs bucket to read files from
            str dest_folder_name: name of the destination folder name to write files to
            str source_folder_name: name of the source folder name to read files from
            str request_file_prefix: prefix of the request file name
            list mime_types: list of accepted mime_types
            str prompt_text: prompt for Gimini
            float temperature: Gimini temprature
            float top_p: Gimini top_p
            float top_k: Gimini top_k
            int max_request_per_file: max number of requests per batch
            int max_output_tokens: Gimini max_output_tokens          
            str video_metadata_file: name of video metadata

        Returns:
            int : number of generated files
          
    """       

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder_name)  
    version=0
    index=0     
    max_index=max_request_per_file

    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    now=datetime.strftime(now, '%Y%m%d%H%M%S')
    versions=[]    

    # Instantiate a Google Cloud Storage client and specify required bucket and file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(dest_bucket_name)
    blob = bucket.blob(video_metadata_file)

    # Download the contents of the blob as a string and then parse it using json.loads() method
    data = json.loads(blob.download_as_string(client=None))
    video_metadata=data['items']
    

    segments_to_process=120 #segments duration
    intervals=120#intervals
    video_start =0 #where from video to start

    for blob in blobs:                         
        if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         metadata=list(filter(lambda element: element['gcs_uri'] ==gcsuri, video_metadata))[0]
                         video_duration=metadata["videoOriginalDurationSecond"]
                         cache_id=metadata["name"]

                         prev=video_start
                         for val in range (segments_to_process,video_duration+segments_to_process,segments_to_process):
                                offset={'start':prev, 'end':val}
                                
                                startOffset=offset['start']
                                endOffset=offset['end']
                                if endOffset>=video_duration:
                                     endOffset=video_duration
                                print(offset)
                                prev=val 
                                segment_prompt= "Only consider video from" + str(startOffset)+" seconds to "+ str(endOffset)+" seconds. Ignore analyzing the rest of video.\n" 
                                if index==0:
                                    request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                                    rf= open(request_file.name, "a") 
                                
                                 
                                request_list=[
                                       json.dumps(
                                              {
                                                "request": 
                                                     {
                                                      "cached_content": cache_id,
                                                      "contents":  {
                                                                        "parts": [
                                                                            {
                                                                            "fileData":  {"fileUri": gcsuri, "mimeType": mimeType},
                                                                            "videoMetadata": {
                                                                                "endOffset": {
                                                                                "nanos": 0,
                                                                                "seconds": endOffset
                                                                                },
                                                                                "startOffset": {
                                                                                "nanos": 0,
                                                                                "seconds": startOffset
                                                                                }
                                                                            }

                                                                            },
                                                                            {
                                                                            "text": prompt_text +"\n"+ segment_prompt
                                                                            } 

                                                                        ],
                                                                        "role": "user"
                                                                        }
                                                          , 
                                                          "generation_config": 
                                                               {"max_output_tokens": max_output_tokens, 
                                                                "temperature":temperature, 
                                                                 "top_k": top_k, 
                                                                 "top_p": top_p
                                                                }
                                                          , 
                                                          "safety_settings": 
                                                           [
                                                            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                                             "threshold": "BLOCK_NONE"
                                                             }
                                                           , 
                                                           {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                                           "threshold": "BLOCK_NONE"
                                                           },
                                                           {"category": "HARM_CATEGORY_HARASSMENT", 
                                                           "threshold": "BLOCK_NONE"
                                                           }, 
                                                           {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                            "threshold": "BLOCK_NONE"
                                                            }


                                                           ]
                                                     }
                                              }
                                       )  +"\n"
                                 ]

                                rf.writelines(request_list)
                                rf.flush()

                                if index==(max_index-1):
                                        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                        rf.close() 
                                        versions.append(version)                               
                                        index=0
                                        version +=1
                                        request_list=[]
                                        rf=None

                                else:                               
                                        index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   
 
    return len(versions)

def create_image_request_file( dest_bucket_name: str= None, source_bucket_name: str= None, source_folder_name: str=None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None, max_request_per_file: int =None):

    """create batch request  file(s) of up to 30000 for videos and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str source_bucket_name: name of the source  gcs bucket to read files from
            str dest_folder_name: name of the destination folder name to write files to
            str source_folder_name: name of the source folder name to read files from
            str request_file_prefix: prefix of the request file name
            list mime_types: list of accepted mime_types
            str prompt_text: prompt for Gimini
            float temperature: Gimini temprature
            float top_p: Gimini top_p
            float top_k: Gimini top_k
            int max_request_per_file: max number of requests per batch
            int max_output_tokens: Gimini max_output_tokens          

        Returns:
            int : number of generated files
          
    """           
         

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder_name)  
    version=0
    index=0     
    max_index=max_request_per_file

    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    now=datetime.strftime(now, '%Y%m%d%H%M%S')
    versions=[]

    for blob in blobs:                         
                    if blob.content_type in mime_types:                            
                         gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                         mimeType=blob.content_type
                         if index==0:
                            request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
                            rf= open(request_file.name, "a") 
                 
                         request_list=[
                               json.dumps(
                                      {
                                        "request": 
                                             {
                                              "contents": 
                                                  {"parts": [{ "fileData": 
                                                                 {"fileUri": gcsuri, "mimeType": mimeType}
                                                              }, 
                                                              {"text": prompt_text
                                                              }
                                                            ]
                                                              , 
                                                    "role": "user"
                                                  }
                                                  , 
                                                  "generation_config": 
                                                       {"max_output_tokens": max_output_tokens, 
                                                        "temperature":temperature, 
                                                         "top_k": top_k, 
                                                         "top_p": top_p
                                                        }
                                                  , 
                                                  "safety_settings": 
                                                   [
                                                    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                                     "threshold": "BLOCK_NONE"
                                                     }
                                                   , 
                                                   {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                                   "threshold": "BLOCK_NONE"
                                                   },
                                                   {"category": "HARM_CATEGORY_HARASSMENT", 
                                                   "threshold": "BLOCK_NONE"
                                                   }, 
                                                   {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                                    "threshold": "BLOCK_NONE"
                                                    }


                                                   ]
                                             }
                                      }
                               )  +"\n"
                         ]

                         rf.writelines(request_list)
                         rf.flush()
 
                         if index==(max_index-1):
                                upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version, request_file_post_fix=now)
                                rf.close() 
                                versions.append(version)                               
                                index=0
                                version +=1
                                request_list=[]
                                rf=None

                         else:                               
                                index =index+1

    if not rf is None: 
        upload_file(rf,dest_bucket_name=dest_bucket_name,request_file_folder=request_file_folder,request_file_prefix=request_file_prefix,version=version,request_file_post_fix=now)
        versions.append(version)   
 
    return len(versions)

@functions_framework.http
def create_batch_request_file(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
    try:
        request_json = request.get_json(silent=True)
        request_args = request.args

        dest_bucket_name =request_args['destination_bucket']
        source_bucket_name =request_args['source_bucket']
        source_folder_name=request_args['source_folder']
        request_file_prefix =request_args['request_file_prefix']
        request_file_folder =request_args['request_file_folder']
        prompt_text= request_args['prompt_text']
        media_types= [media.strip() for media in  str(request_args['media_types']).strip().replace("[",''). replace(']','').replace("'",'').split(',')]


        request_content= request_args['request_content']

        if request_args and 'video_metadata_file' in request_args:
            video_metadata_file= request_args['video_metadata_file']
        else:
          video_metadata_file=""

        if request_args and 'temperature' in request_args:
            temperature= request_args['temperature']
        else:
          temperature=1

        if request_args and 'max_output_tokens' in request_args:
           max_output_tokens= request_args['max_output_tokens'] 
        else:
             max_output_tokens=8192

        if request_args and 'top_p' in request_args:
            top_p= request_args['top_p']
        else:
             top_p=0.95

        if request_args and 'top_k' in request_args:
            top_k= request_args['top_k']
        else:
             top_k=40

        if request_args and 'max_request_per_file' in request_args:
            max_request_per_file= request_args['max_request_per_file']
        else:
          max_request_per_file=30000
    except:
            dest_bucket_name ='artifacts-nine-quality-test-embeddings' #request_args['destination_bucket']
            source_bucket_name ='raw_nine_files'# request_args['source_bucket']
            request_file_prefix = "video_request"#request_args['request_file_prefix']
            request_file_folder =  "video_batch_request_fldr"#request_args['request_file_folder']
            prompt_text= "You are an assistant tasked with summarizing images for retrieval. \\ These summaries will be embedded and used to retrieve the raw image. \\ Give a concise summary of the image that is well optimized for retrieval.\\ If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\\ If you are not sure about any info, please do not make it up. Do not add any extra text to the output."#request_args['prompt_text']
            media_types= ['video/mp4']#list(request_args['media_type'])
            request_content= "video"#request_args['request_content']
            source_folder_name="vlt_video_extract/OTHERS"
            temperature=0.5
            max_output_tokens=2048
            top_p=50
            top_k=0.5
            max_request_per_file=30000
            video_metadata_file="vlt_video_metadata_fldr/vlt_video_metadata.json"

 

    versions=0
    if  request_content=='image':
      versions=create_image_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k, max_request_per_file=max_request_per_file
                                      )  
    if  request_content=='video':
      versions=create_video_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,source_folder_name=source_folder_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k, max_request_per_file=max_request_per_file,
                                     video_metadata_file=video_metadata_file
                                      ) 

    return {"status":"SUCCESS","file_count":versions}


In [106]:
create_batch_request_file('')

{'start': 0, 'end': 120}
{'start': 120, 'end': 240}
{'start': 240, 'end': 360}
{'start': 360, 'end': 480}
{'start': 480, 'end': 600}
{'start': 600, 'end': 720}
{'start': 720, 'end': 840}
{'start': 840, 'end': 960}
{'start': 960, 'end': 1080}


{'status': 'SUCCESS', 'file_count': 1}

In [ ]:
contents = [
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2312.11805v3.pdf",
        mime_type="application/pdf",
    ),
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2403.05530.pdf",
        mime_type="application/pdf",
    ),
]

cached_content = caching.CachedContent.create(
    model_name="gemini-1.5-pro-002",
    system_instruction=system_instruction,
    contents=contents,
    
    display_name="example-cache",
)

print(cached_content.name)

7323099284709048320


In [1]:
import vertexai

from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview import caching

# TODO(developer): Update and un-comment below lines
# PROJECT_ID = "your-project-id"
# cache_id = "your-cache-id"

 
cached_content = caching.CachedContent(cached_content_name="7323099284709048320")

model = GenerativeModel.from_cached_content(cached_content=cached_content)

response = model.generate_content("What are the papers about?")

print(response.text)
# Example response:
# The provided text is about a new family of multimodal models called Gemini, developed by Google.
# ...

The first paper, "Gemini: A Family of Highly Capable Multimodal Models", introduces Gemini 1.0, Google's family of multimodal models. It details their architecture, training, capabilities, and responsible deployment.  Gemini 1.0 models come in different sizes (Ultra, Pro, Nano) for various applications.  The paper highlights Gemini Ultra's state-of-the-art performance on many benchmarks, exceeding human expert performance on MMLU and achieving strong results in multimodal reasoning (like MMMU) and coding tasks.  The paper emphasizes responsible AI development, outlining impact assessments, safety mitigations, and evaluations to address potential harms.

The second paper, "Gemini 1.5: Unlocking multimodal understanding across millions of tokens of context", introduces Gemini 1.5 Pro, an evolution focusing on significantly expanded context length and efficiency.  It details the model's architecture as a mixture-of-experts model and the infrastructure advancements enabling its capabilitie

In [11]:
dir

AttributeError: 'GenerativeModel' object has no attribute 'generation_config'

In [ ]:
 url: ${MM_LLM_ENDPOINT}
                                auth:
                                    type: OAuth2
                                body:
                                    "instances": [
                                                    {
                                                        "image": {
                                                         mimeType:  "${item.mime_type}",
                                                        "gcsUri": "${item.gcs_uri}"  
                                                        }

                                                     } ]
                            result: llm_response

In [29]:
MM_MODEL= "multimodalembedding@001"
MM_METHOD= "predict"
PROJECT='nine-quality-test'
LOCATION='us-central1'
MM_LLM_ENDPOINT="https://" + 'us-central1' + "-aiplatform.googleapis.com" + "/v1/projects/" + PROJECT + "/locations/" + 'us-central1' + "/publishers/google/models/" + MM_MODEL+":"+MM_METHOD
 

In [20]:
import requests

# The API endpoint
url = "https://jsonplaceholder.typicode.com/posts"

# Data to be sent
data = {
     "instances": [
                                                    {
                                                        "image": {
                                                         "mimeType":  "image/png",
                                                        "gcsUri": "gs://raw_nine_files/2023/1/a/816f62dz.png"  
                                                        }

                                                     } ]
}

# A POST request to the API
response = requests.post(MM_LLM_ENDPOINT, json=data)

# Print the response
print(response.json())

{'error': {'code': 401, 'message': 'Request is missing required authentication credential. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.', 'status': 'UNAUTHENTICATED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'CREDENTIALS_MISSING', 'domain': 'googleapis.com', 'metadata': {'method': 'google.cloud.aiplatform.v1.PredictionService.Predict', 'service': 'aiplatform.googleapis.com'}}]}}


In [25]:

from google.cloud import aiplatform
api_regional_endpoint= "us-central1-aiplatform.googleapis.com"
client_options = {"api_endpoint": api_regional_endpoint}
client = aiplatform.gapic.PredictionServiceClient(
            client_options=client_options
        )

In [30]:

instances=[
                                                    {
                                                        "image": {
                                                         "mimeType":  "image/png",
                                                        "gcsUri": "gs://raw_nine_files/2023/1/a/816f62dz.png"  
                                                        }

                                                     } ]

endpoint = (
           f"projects/{PROJECT}/locations/{LOCATION}"
           "/publishers/google/models/multimodalembedding@001"
        )
response =client.predict(endpoint=endpoint, instances=instances)
response.predictions[0].get("imageEmbedding", [])

In [61]:
content_method= "generateContent"
content_model= "gemini-1.5-pro-002"
endpoint=("v1beta1/projects/"  + PROJECT+"/locations/" + LOCATION + "/publishers/google/models/"+ content_model + ":" + content_method)
 
instances={
                                       
                                        "contents": [
                                            {
                                            "role": "user",
                                            "parts": [
                                                {
                                              
                                                
                                                "fileData": {
                                                    "mimeType": "video/mp4",
                                                    "fileUri": "gs://raw_nine_files/vlt_video_extract/MAAT/MAAT2024_1_A_HBB.mp4"
                                                },

                                                "videoMetadata": {
                                                    "startOffset": {
                                                    "seconds": 300,
                                                    "nanos": 0
                                                    },
                                                    "endOffset": {
                                                    "seconds": 520,
                                                    "nanos": 0
                                                    }
                                                }
                                                },
                                                 { "text": "describe what is happening in this video in the given time frame from offset 300 second to offset 520 second in details and what people are talking about."}
                                            ]
                                            }
                                        ] ,
                                        
                                        "safetySettings": [
                                            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"}
                                        ],
                                        "generationConfig": {
                                            "temperature": 1,
                                            "topP": 0.5,
                                            "topK": 40,
                                           # "candidateCount": integer,
                                            "maxOutputTokens": 8192,
                                            # "presencePenalty": float,
                                            # "frequencyPenalty": float,
                                            # "stopSequences": [
                                            # string
                                            # ],
                                            # "responseMimeType": string,
                                            # "responseSchema": schema,
                                            # "seed": integer,
                                            # "responseLogprobs": boolean,
                                            # "logprobs": integer,
                                            # "audioTimestamp": boolean
                                        }#,

                                       # ,"cachedContent": "projects/494586852359/locations/us-central1/cachedContents/4906355134671355904"  
                                       
} 

from google.cloud import aiplatform
api_regional_endpoint= "us-central1-aiplatform.googleapis.com"
client_options = {"api_endpoint": api_regional_endpoint}
client = aiplatform.gapic.PredictionServiceClient(
            client_options=client_options
        )

import google.cloud.aiplatform.v1beta1  as endpoint       
#response =client.generate_content(instances)
#response =client.predict(endpoint=endpoint, instances=[instances])

In [60]:
 dir(endpoint.schema.predict.prediction_v1beta1)

['ClassificationPredictionResult',
 'ImageObjectDetectionPredictionResult',
 'ImageSegmentationPredictionResult',
 'TabularClassificationPredictionResult',
 'TabularRegressionPredictionResult',
 'TextExtractionPredictionResult',
 'TextSentimentPredictionResult',
 'TimeSeriesForecastingPredictionResult',
 'VideoActionRecognitionPredictionResult',
 'VideoClassificationPredictionResult',
 'VideoObjectTrackingPredictionResult',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'gapic_version',
 'package_version',
 'types']

In [62]:
import google.auth
import google.auth.transport.requests
creds, project = google.auth.default()

# creds.valid is False, and creds.token is None
# Need to refresh credentials to populate those

auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)

In [68]:
import base64
import time
import typing
import math
import numpy as np

from google.cloud import aiplatform
from google.protobuf import struct_pb2

#libraries to generate image summaries
from vertexai.vision_models import Video
from vertexai.vision_models import VideoSegmentConfig
from vertexai.vision_models import MultiModalEmbeddingModel
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import Image as vision_model_Image
from vertexai.preview.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part as GenerativeModelPart,
    HarmBlockThreshold,
    HarmCategory,
)
from typing import Any, Dict, List, Literal, Optional, Union

from moviepy.editor import VideoFileClip
 
import requests
from google.cloud import storage
from google.cloud.aiplatform import initializer as aiplatform_initializer
import datetime

import yaml
import re

if 1==1:
         generative_multimodal_model= GenerativeModel("gemini-1.5-flash-001")
         #generation_config= GenerationConfig(temperature=1, top_k=40,top_p=0.95,max_output_tokens=8192) 
         generation_config=GenerationConfig(temperature=1, top_k=40,top_p=0.95,max_output_tokens=8192)#, response_mime_type='application/json',
	     # response_schema=json.loads(schema))  
         #for video, BLOCK_NONE gives error. So, have to set it to BLOCK_ONLY_HIGH
         safety_settings=  {
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    }
         stream=False
        
         video_duration=2885#math.ceil(get_video_duration(video_file))
         
         video_chapters=[]
         prev=0
         log=[]
        segments_to_process=120
         
         for val in range (segments_to_process,video_duration+segments_to_process,segments_to_process):
                offset={'start':prev, 'end':val}
                prev=val    
                print(offset)
                startOffset=offset['start']
                endOffset=offset['end']
                chapters=[]  
                
                if endOffset>=video_duration:
                    endOffset=video_duration
                    
                video_description_prompt=f"""You are an assistant tasked with summarizing videos for retrieval.\
                     These summaries will be embedded and used to retrieve the raw video.\
                    Chapterize the video content by grouping the video content into chapters \
                    with intervals of {intervals} seconds and providing a concise detail for each chapter that is well optimized for retrieval.\
                    If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\
                    Describe important scenes in the video concisely.\
                    If you are not sure about any info, please do not make it up. \
                    Only consider video from {startOffset} seconds to {endOffset} seconds. Ignore analyzing the rest of video.\
                    If it is the last chapter, set the endOffset to {endOffset} instead.\ 
                    If a chapter includes prohibited content, set chapterSummary to "".\
                    For result, follow JSON schema.<JSONSchema>{json.dumps(schema)}</JSONSchema>"
                    """   
 
        
                contents=[parts=[GenerativeModelPart.from_uri(video_file,mime_type="video/mp4", ),
                           video_description_prompt,]        

                model_response = generative_multimodal_model.generate_content(
                                    contents,
                                    generation_config=generation_config,
                                    stream=stream,
                                    safety_settings=safety_settings, )        

                response_list = []
                    # print(model_response)

                print('Called the API, processing the result now...')
                prohibited_content=False
                    

NameError: name 'video_start' is not defined

In [71]:
help(generative_multimodal_model.generate_content)

Help on method generate_content in module vertexai.generative_models._generative_models:

generate_content(contents: Union[List[ForwardRef('Content')], List[Dict[str, Any]], str, ForwardRef('Image'), ForwardRef('Part'), List[Union[str, ForwardRef('Image'), ForwardRef('Part')]]], *, generation_config: Union[ForwardRef('GenerationConfig'), Dict[str, Any], NoneType] = None, safety_settings: Union[List[ForwardRef('SafetySetting')], Dict[google.cloud.aiplatform_v1beta1.types.content.HarmCategory, google.cloud.aiplatform_v1beta1.types.content.SafetySetting.HarmBlockThreshold], NoneType] = None, tools: Optional[List[ForwardRef('Tool')]] = None, tool_config: Optional[ForwardRef('ToolConfig')] = None, stream: bool = False) -> Union[ForwardRef('GenerationResponse'), Iterable[ForwardRef('GenerationResponse')]] method of vertexai.preview.generative_models.GenerativeModel instance
    Generates content.
    
    Args:
        contents: Contents to send to the model.
            Supports either a li

In [72]:
from oauth2client.client import GoogleCredentials

try:
    creds = GoogleCredentials.get_application_default()
except Exception as e:
    quit(e)

print("Access Token:", creds.get_access_token().access_token)

Access Token: ya29.c.c0ASRK0GazjrHy8T5oxOjgzmUAvpcPg6TrD2uCutKDaCN3dLuPEdjFLtvwDqGKVErmiHO-4iC92m7j-813QysLmaxrDYuXiRI07YF4Fb4YEJx5d8ignk4uyiWgPd7WZnO9CqjQ8HxSB7ZEqZEFuz_WmtVXeSGLyccCDAmgokqzj8CbwV5o0q2Cv2A6R9SMfoLcuYUL7HzUK5QZZLIwb14-cpwKzJzXj-l8UyxGm2Qz2hlhczUKMl--WAIkOmGODPKDCOpdSajsuGRbrKpw13MKrDuZl2HITFRE0nfF1pwClYlqTgt2H_V8OtIN8YYsvNAl-Z3f0bimuUDiA095RSnPipdSqFCN3ECwJrHa-iOmXU9SZhcvQJF7Bg_rrR0ANFRmzKCjuIDly_mHBtgqjjRVH413AshIg25-IsyVXvcg3zxUBkFyJdWI8OwZOSlt8y3Rx95FZsJ8cS7SinQOo2z5wX8Xl832FYb7Jlf1jf-jOUhgraMIk6qget7VmyiJ1me6RV82FMt2mqgx5hrSzQ0o34_03zFvwe2ZpQv0r78JBjZpg9wflsJd6ZqkQqFui8207ozoqRzStc4YU9157lzs3pagOxhV3Ju1OMp32ZROwc97hBys2tSj4ZRiFecXqFYIejMYX399dlZ-qkOrdlS0yqyk3iX3jcIOUox5-rqrOXbB8qh6hrUc4p9uv_WgvMUpYUqii85-4VmXU_czOZ7RYfYoFR6Xm73ZQrMVac3BVkSv73O5vMOrboZFazhcuzJx8c8v458i8nb336m9fdp5BXma4RpiUXVrUQcg7fi0YYJgRv6wcdd78wVBrVbxtMbr0QgZ5ItIBqprY7yv8t6Uc4kcQcMdv7MykUrI_1ObV-5Z9_QgFnt7Z9ouSBnyjyOypetS1hgunf6rU9eQY6U57f16mI-wlhqvJke__mwgkxWoyF0pml9kfrYttXyJZprSb3_wmy1pn95BW-35c

In [1]:
import functions_framework
from datetime import datetime
import logging
from google.cloud import bigquery
import os, json
from google.cloud import storage

def create_dataset(project_id,dataset_id,region_id):
    """
        Create a dataset
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str region_id:  name of the region under whcih the dataset should be created.               
    """
    
    client = bigquery.Client(project_id)
    print(project_id)
    # Check if the dataset exists
    try:
        dataset = client.get_dataset(f"{project_id}.{dataset_id}")  # Make an API request.
        print(f"Dataset '{dataset_id}' already exists.")
    except :
        # If the dataset does not exist, create it
        
        dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
        dataset.location = region_id # Set your desired location

        # Create the dataset
        dataset = client.create_dataset(dataset)  # Make an API request.
        print(f"Dataset '{dataset_id}' created successfully.")
        
 
def create_table(project_id,dataset_id,table_id):
    
    """
        Create a table 
        
        Args:
           str project_id: project id
           str dataset_id: name of the dataset under which the table should be created.
           str table_id:  name of the table.
        Returns:
            list[bigquery.SchemaField] schema: table schema- list of columns
             
    """
        
    
    client = bigquery.Client(project_id)
    
    full_table_id = f"{project_id}.{dataset_id}.{table_id}"

    # Define the table schema
    schema = [
        bigquery.SchemaField("idx", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("name", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("mime_type", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("gcs_uri", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("media_name", "STRING", mode="REQUIRED")
       
    ]     

    # Check if the table exists
    try:
        table = client.get_table(full_table_id)  # Make an API request.
        print(f"Table '{full_table_id}' already exists.")
        
        # Drop the table if exist
        query = f"DROP TABLE `{full_table_id}`"

        # Execute the query
        try:
            client.query(query).result()  # Make an API request.
            print(f"Table '{full_table_id}' dropped successfully.")
        except Exception as e:
            print(f"Error dropping table '{full_table_id}': {e}")
    except :
        # If the table does not exist, create it
        table = bigquery.Table(full_table_id, schema=schema)

        # Create the table
        table = client.create_table(table)  # Make an API request.
        print(f"Table '{full_table_id}' created successfully.")
        
    return schema


@functions_framework.http
def get_gcs_info(request):
    """
        loads gcs metadata info into big query table
             
    """
    status=''  
    #request_args = request.args
 

#     project_id=  request_args['project_id']
#     dataset_id=  request_args['dataset']
#     table= request_args['table']
#     region= request_args['region']
#     source_bucket= request_args['source_bucket']
#     source_folder= request_args['source_folder']
#     media_types= [media.strip() for media in  str(request_args['media_types']).strip().replace("[",''). replace(']','').replace("'",'').split(',')]

            
    project_id=  "nine-quality-test"
    dataset_id= 'vlt_metadata'
    table= "vlt_image_metadata"
    region= "us-central1"
    source_bucket= 'raw_nine_files'
    source_folder= "2024"
    media_types= ['image/png']
    
            

    # Initialize a storage client
    storage_client = storage.Client()
    # Get the bucket
    bucket = storage_client.get_bucket(source_bucket)
    # List all objects in the bucket
    blobs = bucket.list_blobs(prefix=source_folder)
    
    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()  
    rows_to_insert=[]        
    client = bigquery.Client(project_id)    
    #create data set if does not exist
    create_dataset(project_id,dataset_id,region)


    job_list=[]
    idx=0
    for blob in blobs:
        if blob.content_type in media_types:
            rows_to_insert.append(
                                {   "idx":  idx  , 
                                    "name": blob.name, 
                                    "mime_type":blob.content_type,
                                    "gcs_uri":  "gs://"+source_bucket+"/"+blob.name,
                                    "media_name":blob.name.split("/")[-1].replace("."+blob.content_type.split("/")[-1],"")
                                    }
                                            )
        
            idx=idx+1
            
    print(rows_to_insert)
    #create table new if does not exist
    table=f"{table}" 
    table_schema=create_table(project_id,dataset_id,table)
    #push the data into the table
    table_id = f"{project_id}.{dataset_id}.{table}"
    dataset  = client.dataset(dataset_id)
    table = dataset.table(table)
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    job_config.schema = table_schema
    job = client.load_table_from_json(rows_to_insert, table, job_config = job_config)                
    job_list.append(job.job_id)
  
    return {'status':'SUCCESS', 'record_count':idx,'count_of_tables':1,'table_name_prefix':table,'jobs':job_list }

In [2]:
get_gcs_info('')

nine-quality-test
Dataset 'vlt_metadata' already exists.
[{'idx': 0, 'name': '2024/3/d/screenshot_directrycontent.png', 'mime_type': 'image/png', 'gcs_uri': 'gs://raw_nine_files/2024/3/d/screenshot_directrycontent.png', 'media_name': 'screenshot_directrycontent'}]
Table 'nine-quality-test.vlt_metadata.vlt_image_metadata' created successfully.


{'status': 'SUCCESS',
 'record_count': 1,
 'count_of_tables': 1,
 'table_name_prefix': TableReference(DatasetReference('nine-quality-test', 'vlt_metadata'), 'vlt_image_metadata'),
 'jobs': ['acdf507c-f282-4d3f-bce7-89aad5c12d4d']}

In [80]:
table_schema

NameError: name 'table_schema' is not defined

In [12]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define your SQL CREATE MODEL statement
create_model_sql = """CREATE MODEL `vlt_metadata.multimodal`
REMOTE WITH CONNECTION `us-central1.vlt_multimodal_endpoint`
OPTIONS(endpoint = 'multimodalembedding@001');
"""

# Execute the CREATE MODEL statement
query_job = client.query(create_model_sql)

# Wait for the job to complete
query_job.result()

print("Model created successfully.")

Model created successfully.


SyntaxError: invalid syntax (1852933557.py, line 1)

In [25]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define your SQL CREATE MODEL statement
create_model_sql = """CREATE OR REPLACE EXTERNAL TABLE `vlt_metadata.met_images`
WITH CONNECTION `us-central1.vlt_multimodal_endpoint`
OPTIONS
  ( object_metadata = 'SIMPLE',
    uris = ['gs://raw_nine_files/2023/*']
    )
"""

 
    

# Execute the CREATE MODEL statement
query_job = client.query(create_model_sql)

# Wait for the job to complete
query_job.result()

print("Model created successfully.")

Model created successfully.


In [30]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define your SQL CREATE MODEL statement
create_model_sql = """CREATE OR REPLACE TABLE `vlt_metadata.met_image_embeddings`
AS
SELECT *
FROM
  ML.GENERATE_EMBEDDING(
    MODEL `vlt_metadata.multimodal`,
    (SELECT  * FROM `nine-quality-test.vlt_metadata.met_images` LIMIT 1000));
"""


    

# Execute the CREATE MODEL statement
query_job = client.query(create_model_sql)

# Wait for the job to complete
query_job.result()

print("Model created successfully.")

TypeError: Invalid job configuration received.

In [32]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define your SQL CREATE MODEL statement
create_model_sql = """CREATE OR REPLACE TABLE `vlt_metadata.met_image_embeddings1`
AS
SELECT *
FROM
  ML.GENERATE_EMBEDDING(
    MODEL `vlt_metadata.multimodal`,
    (SELECT  * FROM `nine-quality-test.vlt_metadata.met_images` LIMIT 1000));
"""


    

# Execute the CREATE MODEL statement
query_job = client.create_job( job_config={
        "query": {
            "query": create_model_sql,
        },
        "labels": {"example-label": "example-value"},
        "maximum_bytes_billed": 10000000,
    })

# Wait for the job to complete
query_job.result()

print("Model created successfully.")

Model created successfully.


In [35]:
query_job.job_id

'bd806266-d3e0-4264-b7f1-737ece17ae8d'